- In this Notebook, I am trying to integrate the training data from the backscore and rest of the period from prj-prod-dataplatform.audit_balance.ml_model_run_details table.
- In this I will compare the training period with each month of test period.

**Steps to Follow**:

* Read the specific model data from prj-prod-dataplatform.audit_balance.ml_model_run_details table
* Expand the calcFeature column to extract all the features for the model
* Read the data from specific backscore table for the training data
* Identify the features and create a list
* Use transform_data function to create the same structure as ml_model_run_details table
* Insert the data to a similar training table - prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
* Read the specific model data from prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
* expand the training set from the calcFeature column
* Concatenate both the test and train datasets
* Calculate the PSI using the PSI function comparing it with the train set
* Insert the result to a PSI table prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_psi_v4

# **PSI - CSI Calculation**

## Define Libraries

In [3]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

### Function

#### expand_calc_features

In [4]:
import pandas as pd
import json

def expand_calc_features(df):
    """
    Expand the calcFeatures JSON column into separate columns and return the complete DataFrame.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_list = []

    for idx, calc_features_str in enumerate(df['calcFeatures']):
        try:
            # Parse the JSON string
            features_dict = json.loads(calc_features_str.replace("'", '"'))  # Replace single quotes with double quotes for valid JSON
            calc_features_list.append(features_dict)
        except (json.JSONDecodeError, AttributeError) as e:
            # If parsing fails, create an empty dict and print warning
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            calc_features_list.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_list)

    # Add prefix to JSON-derived columns to avoid conflicts
    calc_features_df = calc_features_df.add_prefix('calc_')

    # Reset index to ensure proper alignment
    df_expanded = df_expanded.reset_index(drop=True)
    calc_features_df = calc_features_df.reset_index(drop=True)

    # Combine original DataFrame with expanded calcFeatures
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    return result_df


#### expand_calc_features_robust

In [5]:
import pandas as pd
import json

def expand_calc_features_robust(df):
    """
    Expand the calcFeatures JSON column into separate columns with better error handling.

    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data

    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """

    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()

    # Parse the calcFeatures JSON column
    calc_features_data = []

    for idx, row in df.iterrows():
        calc_features_str = row['calcFeatures']

        if pd.isna(calc_features_str) or calc_features_str == '':
            calc_features_data.append({})
            continue

        try:
            # Clean the string and parse JSON
            cleaned_str = calc_features_str.replace("'", '"').replace('None', 'null').replace('True', 'true').replace('False', 'false')
            features_dict = json.loads(cleaned_str)
            calc_features_data.append(features_dict)
        except Exception as e:
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            print(f"Problematic string: {calc_features_str[:100]}...")  # Print first 100 chars
            calc_features_data.append({})

    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_data)

    # Add prefix to JSON-derived columns to avoid conflicts with existing columns
    calc_features_df = calc_features_df.add_prefix('feat_')

    # Combine DataFrames
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)

    print(f"Original DataFrame shape: {df.shape}")
    print(f"Expanded DataFrame shape: {result_df.shape}")
    print(f"Added {len(calc_features_df.columns)} new columns from calcFeatures")

    return result_df

#### transform_data

In [6]:
# import pandas as pd
# import json
# import uuid
# from datetime import datetime
# from typing import List

# def transform_data(d1: pd.DataFrame, feature_column: List[str], a='demo_score', modelDisplayName = 'Cash_beta_trench1_Demo_backscore', subscription_name = 'sil_march 25 models'):
#     # Read the input CSV file
#     df = d1.copy()
    
#     # Create the output DataFrame with the required structure
#     output_data = []
    
#     for _, row in df.iterrows():
#         # Create the calcFeature JSON with all the feature columns
#         feature_columns = feature_column
        
#         calc_feature = {}
#         for col in feature_columns:
#             if col in row and pd.notna(row[col]):
#                 # Convert Timestamp objects to string
#                 if isinstance(row[col], pd.Timestamp):
#                     calc_feature[col] = row[col].isoformat()
#                 else:
#                     calc_feature[col] = row[col]
        
       
#         # Get current timestamp
#         current_time = datetime.now().isoformat()
        
#         # Create the output row
#         output_row = {
#             "customerId": row['customer_id'],
#             "digitalLoanAccountId": row['digitalLoanAccountId'],
#             "crifApplicationId": str(uuid.uuid4()),  # Generate random UUID
#             "prediction": row.get(a, 0),
#             "start_time": current_time,
#             "end_time": current_time,
#             "modelDisplayName":modelDisplayName,
#             "modelVersionId":"v1",
#             "subscription_name": subscription_name,
#             "message_id": str(uuid.uuid4()),  # Generate random UUID
#             "publish_time": current_time,
#             "attributes": "{}",  # Empty JSON object
#             "calcFeature": json.dumps(calc_feature, default=str)  # Use default=str to handle non-serializable objects
            
#         }
        
#         output_data.append(output_row)
    
#     # Create DataFrame from the output data
#     output_df = pd.DataFrame(output_data)
    
#     return output_df

# # Example usage:
# # transformeddata = 'cash_beta_trench1_applied_loans_backscored_20241001_20250831'
# # transform_data(f'{LOCALPATH}/{transformeddata}.csv')

import pandas as pd
import json
import uuid
from datetime import datetime
from typing import List

def transform_data(
    d1: pd.DataFrame, 
    feature_column: List[str], 
    a: str = 'demo_score', 
    modelDisplayName: str = 'Cash_beta_trench1_Demo_backscore', 
    tc: str = "", 
    subscription_name: str = 'sil_march 25 models'
) -> pd.DataFrame:
    """
    Transforms input data into a structured format suitable for model scoring output.

    Parameters:
    - d1 (pd.DataFrame): Input DataFrame containing raw data.
    - feature_column (List[str]): List of column names to include in the 'calcFeature' JSON.
    - a (str): Column name containing the prediction score. Default is 'demo_score'.
    - modelDisplayName (str): Name of the model used for scoring.
    - tc (str): Trench category (optional).
    - do (str): Device operating system. Default is 'android'.
    - subscription_name (str): Name of the subscription or model group.

    Returns:
    - pd.DataFrame: Transformed DataFrame with structured output.
    """

    # Make a copy of the input DataFrame to avoid modifying the original
    df = d1.copy()
    
    # Initialize an empty list to store transformed rows
    output_data = []
    
    # Iterate over each row in the DataFrame
    for _, row in df.iterrows():
        # Initialize dictionary to hold feature values
        calc_feature = {}
        
        # Loop through each feature column and extract its value from the row
        for col in feature_column:
            if col in row and pd.notna(row[col]):
                # Convert datetime values to ISO format strings
                if isinstance(row[col], pd.Timestamp):
                    calc_feature[col] = row[col].isoformat()
                else:
                    calc_feature[col] = row[col]
        
        # Get the current timestamp for start_time, end_time, and publish_time
        current_time = datetime.now().isoformat()
        
        # Construct the output row dictionary with required fields
        output_row = {
            "customerId": row['customer_id'],  # Unique customer identifier
            "digitalLoanAccountId": row['digitalLoanAccountId'],  # Loan account ID
            "crifApplicationId": str(uuid.uuid4()),  # Random UUID for application ID
            "prediction": row.get(a, 0),  # Prediction score from specified column
            "start_time": current_time,  # Timestamp when processing starts
            "end_time": current_time,    # Timestamp when processing ends
            "modelDisplayName": modelDisplayName,  # Name of the model used
            "modelVersionId": "v1",  # Static model version
            "calcFeature": json.dumps(calc_feature, default=str),  # Features as JSON string
            "subscription_name": subscription_name,  # Subscription name
            "message_id": str(uuid.uuid4()),  # Random UUID for message ID
            "publish_time": current_time,  # Timestamp when message is published
            "attributes": "{}",  # Placeholder for additional attributes
            "trenchCategory": tc,  # Optional trench category
            "deviceOs": row['osType'],  # Device operating system
        }
        
        # Append the transformed row to the output list
        output_data.append(output_row)
    
    # Convert the list of dictionaries to a DataFrame
    output_df = pd.DataFrame(output_data)
    
    # Return the transformed DataFrame
    return output_df



#### PSI Functions new

In [7]:
## Updated on 27-10-2025 - Modified for Training Period Baseline
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

def identify_feature_types(df: pd.DataFrame, feature_list: List[str]) -> Dict[str, List[str]]:
    """
    Identify categorical and numerical features from the feature list.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to classify

    Returns:
    --------
    Dict with 'categorical' and 'numerical' keys containing respective feature lists
    """
    categorical_features = []
    numerical_features = []

    for feature in feature_list:
        if feature not in df.columns:
            print(f"Warning: Feature '{feature}' not found in dataframe")
            continue

        # Check if feature is numeric
        if pd.api.types.is_numeric_dtype(df[feature]):
            # If unique values are less than 15 and all integers, treat as categorical
            unique_vals = df[feature].nunique()
            if unique_vals < 15 and df[feature].dropna().apply(lambda x: x == int(x) if isinstance(x, (int, float)) else False).all():
                categorical_features.append(feature)
            else:
                numerical_features.append(feature)
        else:
            categorical_features.append(feature)

    return {
        'categorical': categorical_features,
        'numerical': numerical_features
    }


def create_bins_for_features(df: pd.DataFrame,
                             numerical_features: List[str],
                             categorical_features: List[str],
                             train_period_df: pd.DataFrame) -> Dict:
    """
    Create bins for numerical features (deciles with fallback) and categorical features (top 6 + others)
    based on the entire training period data.

    Parameters:
    -----------
    df : pd.DataFrame
        Full input dataframe
    numerical_features : List[str]
        List of numerical features
    categorical_features : List[str]
        List of categorical features
    train_period_df : pd.DataFrame
        Training period dataframe (June 2024 to March 2025)

    Returns:
    --------
    Dictionary containing binning information for each feature
    """
    binning_info = {}

    # Create bins for numerical features with fallback strategy
    for feature in numerical_features:
        valid_data = train_period_df[feature].dropna()

        if len(valid_data) == 0:
            binning_info[feature] = {'type': 'numerical', 'bins': None, 'bin_ranges': {}}
            continue

        bins = None
        bin_count = None

        # Try 10 bins (deciles)
        try:
            test_bins = np.percentile(valid_data, np.arange(0, 101, 10))
            test_bins = np.unique(test_bins)
            if len(test_bins) >= 11:  # 11 edges = 10 bins
                bins = test_bins
                bin_count = 10
        except Exception as e:
            pass

        # If 10 bins not possible, try 5 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, np.arange(0, 101, 20))
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 6:  # 6 edges = 5 bins
                    bins = test_bins
                    bin_count = 5
            except Exception as e:
                pass

        # If 5 bins not possible, try 3 bins
        if bins is None:
            try:
                test_bins = np.percentile(valid_data, [0, 33.33, 66.67, 100])
                test_bins = np.unique(test_bins)
                if len(test_bins) >= 4:  # 4 edges = 3 bins
                    bins = test_bins
                    bin_count = 3
            except Exception as e:
                pass

        # If still no bins possible, use equal distance bins of 5
        if bins is None:
            print(f"Warning: Feature '{feature}' has insufficient variance - cannot create standard bins")
            print(f"Feature '{feature}': Using equal distance bins of 5")

            min_val = valid_data.min()
            max_val = valid_data.max()

            # Create 5 equal distance bins
            bins = np.linspace(min_val, max_val, 6)  # 6 edges = 5 bins
            bins = np.unique(bins)
            bin_count = len(bins) - 1

            # If all values are the same, add slight buffer
            if bin_count == 1:
                bins = np.array([min_val - 0.1, min_val, min_val + 0.1])
                bin_count = 2
                print(f"Feature '{feature}': Constant value ({min_val}). Created 2 equal distance bins with buffer")

        # Add infinity edges to capture all values
        bins = bins.copy()
        bins[0] = -np.inf
        bins[-1] = np.inf

        print(f"Feature '{feature}': Created {bin_count} bins")

        # Create bin ranges dictionary
        bin_ranges = {}
        for i in range(len(bins)-1):
            bin_name = f"Bin_{i+1}"
            bin_ranges[bin_name] = {
                'min': bins[i],
                'max': bins[i+1],
                'range_str': f"[{bins[i]:.2f}, {bins[i+1]:.2f}]" if not np.isinf(bins[i]) and not np.isinf(bins[i+1]) else f"({bins[i]}, {bins[i+1]})"
            }

        binning_info[feature] = {
            'type': 'numerical',
            'bins': bins,
            'bin_ranges': bin_ranges,
            'bin_count': bin_count
        }

    # Create bins for categorical features (top 6 + others) using training period
    for feature in categorical_features:
        value_counts = train_period_df[feature].value_counts()
        unique_categories = value_counts.index.tolist()
        print(f"Unique categories: {unique_categories}")

        if len(unique_categories) <= 6:
            # Treat each category as a separate bin
            top_categories = unique_categories
        else:
            # Use top 6 categories only
            top_categories = value_counts.nlargest(6).index.tolist()

        print(f"Top categories for feature '{feature}': {top_categories}")

        binning_info[feature] = {
                'type': 'categorical',
                'top_categories': top_categories,
                'bin_ranges': {}  # No ranges for categorical
            }

    return binning_info


def apply_binning(df: pd.DataFrame,
                  feature: str,
                  binning_info: Dict) -> pd.Series:
    """
    Apply binning to a feature based on binning information.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature : str
        Feature name
    binning_info : Dict
        Binning information for the feature

    Returns:
    --------
    pd.Series with binned values
    """
    if binning_info['type'] == 'numerical':
        if binning_info['bins'] is None:
            return pd.Series(['Missing'] * len(df), index=df.index)

        bins = binning_info['bins']
        labels = [f"Bin_{i+1}" for i in range(len(bins)-1)]

        binned = pd.cut(df[feature],
                       bins=bins,
                       labels=labels,
                       include_lowest=True,
                       duplicates='drop')

        # Handle nulls - convert to string and then replace
        binned = binned.astype(str)
        binned[df[feature].isna()] = 'Missing'

        return binned

    else:  # categorical
        top_cats = binning_info['top_categories']

        # Convert to string for consistent comparison
        if pd.api.types.is_categorical_dtype(df[feature]):
            feature_data = df[feature].astype(str)
        else:
            feature_data = df[feature].astype(str)

        # Replace NaN string representation with 'Missing'
        feature_data = feature_data.replace('nan', 'Missing')

        # Convert top_cats to strings for comparison
        top_cats_str = [str(cat) for cat in top_cats]

        # Apply binning logic: use category name if in top_cats, else 'Others' (except for Missing)
        binned = feature_data.apply(lambda x: x if x in top_cats_str else ('Others' if x != 'Missing' else 'Missing'))

        return binned


def calculate_psi(expected_pct: pd.Series,
                  actual_pct: pd.Series,
                  epsilon: float = 0.0001) -> float:
    """
    Calculate Population Stability Index with proper epsilon handling and renormalization.

    Parameters:
    -----------
    expected_pct : pd.Series
        Expected (baseline) percentages
    actual_pct : pd.Series
        Actual percentages
    epsilon : float
        Small value to avoid log(0)

    Returns:
    --------
    PSI value
    """
    # Align indices
    all_bins = expected_pct.index.union(actual_pct.index)
    expected_pct = expected_pct.reindex(all_bins, fill_value=0)
    actual_pct = actual_pct.reindex(all_bins, fill_value=0)

    # Only add epsilon where values are zero
    expected_pct = expected_pct.apply(lambda x: epsilon if x == 0 else x)
    actual_pct = actual_pct.apply(lambda x: epsilon if x == 0 else x)

    # Renormalize to ensure they sum to 1 after adding epsilon
    expected_pct = expected_pct / expected_pct.sum()
    actual_pct = actual_pct / actual_pct.sum()

    # Calculate PSI
    psi_value = np.sum((actual_pct - expected_pct) * np.log(actual_pct / expected_pct))

    return psi_value


def calculate_month_on_month_psi(df: pd.DataFrame,
                                 feature_list: List[str],
                                 segment_columns: List[str],
                                 month_col: str = 'Application_month',
                                 data_selection_col: str = 'Data_selection',
                                 account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate PSI for each feature comparing training period (June 2024 to March 2025)
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column (identifies train period)
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with PSI values with one row per feature-month-segment combination
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []

    # Calculate overall PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(f"Feature binned {df[f'{feature}_binned']}")
        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate PSI for each test month
        for month in test_months:
            actual_dist = df[df[month_col] == month][f'{feature}_binned'].value_counts(normalize=True)
            psi_value = calculate_psi(train_baseline, actual_dist)

            # Calculate average percentages across all bins
            expected_avg_pct = train_baseline.mean() * 100
            actual_avg_pct = actual_dist.mean() * 100

            # # Count distinct accounts for segment
            # base_segment_count = train_segment[account_id_col].nunique()
            # actual_segment_count = actual_segment[account_id_col].nunique()


            results.append({
                'Feature': feature,
                'Feature_Type': binning_info[feature]['type'],
                'Segment_Column': 'Overall',
                'Segment_Value': 'All',
                'Month': f"{month}",
                'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                'Current_Month': month,
                'Expected_Percentage': expected_avg_pct,
                'Actual_Percentage': actual_avg_pct,
                'PSI': psi_value
            })

    # Calculate PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)
                    psi_value = calculate_psi(train_baseline, actual_dist)

                    # Calculate average percentages across all bins
                    expected_avg_pct = train_baseline.mean() * 100
                    actual_avg_pct = actual_dist.mean() * 100

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    results.append({
                        'Feature': feature,
                        'Feature_Type': binning_info[feature]['type'],
                        'Segment_Column': segment_col,
                        'Segment_Value': segment_val,
                        'Month': f"{month}",
                        'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                        'Current_Month': month,
                        'Base_Count': base_segment_count,
                        'Actual_Count': actual_segment_count,
                        'Expected_Percentage': expected_avg_pct,
                        'Actual_Percentage': actual_avg_pct,
                        'PSI': psi_value
                    })

    return pd.DataFrame(results)


def calculate_bin_level_psi(df: pd.DataFrame,
                            feature_list: List[str],
                            segment_columns: List[str],
                            month_col: str = 'Application_month',
                            data_selection_col: str = 'Data_selection',
                            account_id_col: str = 'digitalLoanAccountId') -> pd.DataFrame:
    """
    Calculate bin-level PSI for each feature comparing training period
    vs each month after March 2025, overall and by segments.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe
    feature_list : List[str]
        List of features to calculate PSI for
    segment_columns : List[str]
        List of segment columns
    month_col : str
        Name of month column
    data_selection_col : str
        Name of data selection column
    account_id_col : str
        Name of account ID column for counting distinct accounts

    Returns:
    --------
    pd.DataFrame with bin-level PSI details including bin ranges
    """
    # Create a copy to avoid modifying original
    df = df.copy()

    # Identify training and test periods
    train_df = df[df[data_selection_col] == 'Train'].copy()
    test_df = df[df[data_selection_col] != 'Train'].copy()

    if len(train_df) == 0:
        raise ValueError("No training data found. Check Data_selection column.")

    print(f"Training period: {train_df[month_col].min()} to {train_df[month_col].max()}")
    print(f"Test period: {test_df[month_col].min()} to {test_df[month_col].max()}")

    # Identify feature types
    feature_types = identify_feature_types(df, feature_list)

    # Create binning strategy based on training period
    binning_info = create_bins_for_features(
        df,
        feature_types['numerical'],
        feature_types['categorical'],
        train_df
    )

    # Get sorted test months
    test_months = sorted(test_df[month_col].unique())

    results = []
    epsilon = 0.0001

    # Calculate overall bin-level PSI
    for feature in feature_list:
        if feature not in df.columns:
            continue

        # Apply binning to entire dataset
        df[f'{feature}_binned'] = apply_binning(df, feature, binning_info[feature])
        # print(df[f'{feature}_binned'])

        # Get training period distribution (baseline)
        train_baseline = df[df[data_selection_col] == 'Train'][f'{feature}_binned'].value_counts(normalize=True)

        # Calculate bin-level PSI for each test month
        for month in test_months:
            month_data = df[df[month_col] == month]
            actual_dist = month_data[f'{feature}_binned'].value_counts(normalize=True)

            # Count distinct accounts
            base_count = df[df[data_selection_col] == 'Train'][account_id_col].nunique()
            actual_count = month_data[account_id_col].nunique()

            # Get all bins
            all_bins = train_baseline.index.union(actual_dist.index)

            for bin_name in all_bins:
                # Simplified epsilon logic - no redundancy
                expected_pct = train_baseline.get(bin_name, 0)
                actual_pct = actual_dist.get(bin_name, 0)

                # Add epsilon only if zero
                expected_pct = epsilon if expected_pct == 0 else expected_pct
                actual_pct = epsilon if actual_pct == 0 else actual_pct

                # Calculate bin-level PSI
                bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                # Get bin range information
                bin_ranges = binning_info[feature]['bin_ranges']
                if bin_name in bin_ranges:
                    bin_min = bin_ranges[bin_name]['min']
                    bin_max = bin_ranges[bin_name]['max']
                    bin_range = bin_ranges[bin_name]['range_str']
                else:
                    # For categorical or special bins (Missing, Others)
                    bin_min = None
                    bin_max = None
                    bin_range = bin_name

                results.append({
                    'Feature': feature,
                    'Feature_Type': binning_info[feature]['type'],
                    'Segment_Column': 'Overall',
                    'Segment_Value': 'All',
                    'Month': f"{month}",
                    'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                    'Current_Month': month,
                    'Base_Count': base_count,
                    'Actual_Count': actual_count,
                    'Bin': bin_name,
                    'Bin_Range': bin_range,
                    'Bin_Min': bin_min,
                    'Bin_Max': bin_max,
                    'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                    'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                    'Bin_PSI': bin_psi
                })

    # Calculate bin-level PSI by segments
    for segment_col in segment_columns:
        if segment_col not in df.columns:
            continue

        segments = df[segment_col].dropna().unique()

        for segment_val in segments:
            segment_df = df[df[segment_col] == segment_val]

            for feature in feature_list:
                if feature not in df.columns:
                    continue

                # Get training period distribution for segment
                train_segment = segment_df[segment_df[data_selection_col] == 'Train']
                if len(train_segment) == 0:
                    continue

                train_baseline = train_segment[f'{feature}_binned'].value_counts(normalize=True)

                # Calculate bin-level PSI for each test month
                for month in test_months:
                    actual_segment = segment_df[segment_df[month_col] == month]
                    if len(actual_segment) == 0:
                        continue

                    actual_dist = actual_segment[f'{feature}_binned'].value_counts(normalize=True)

                    # Count distinct accounts for segment
                    base_segment_count = train_segment[account_id_col].nunique()
                    actual_segment_count = actual_segment[account_id_col].nunique()

                    # Get all bins
                    all_bins = train_baseline.index.union(actual_dist.index)

                    for bin_name in all_bins:
                        # Simplified epsilon logic - no redundancy
                        expected_pct = train_baseline.get(bin_name, 0)
                        actual_pct = actual_dist.get(bin_name, 0)

                        # Add epsilon only if zero
                        expected_pct = epsilon if expected_pct == 0 else expected_pct
                        actual_pct = epsilon if actual_pct == 0 else actual_pct

                        # Calculate bin-level PSI
                        bin_psi = (actual_pct - expected_pct) * np.log(actual_pct / expected_pct)

                        # Get bin range information
                        bin_ranges = binning_info[feature]['bin_ranges']
                        if bin_name in bin_ranges:
                            bin_min = bin_ranges[bin_name]['min']
                            bin_max = bin_ranges[bin_name]['max']
                            bin_range = bin_ranges[bin_name]['range_str']
                        else:
                            # For categorical or special bins (Missing, Others)
                            bin_min = None
                            bin_max = None
                            bin_range = bin_name

                        results.append({
                            'Feature': feature,
                            'Feature_Type': binning_info[feature]['type'],
                            'Segment_Column': segment_col,
                            'Segment_Value': segment_val,
                            'Month': f"{month}",
                            'Base_Month': 'Train (Jun 2024 - Mar 2025)',
                            'Current_Month': month,
                            'Base_Count': base_segment_count,
                            'Actual_Count': actual_segment_count,
                            'Bin': bin_name,
                            'Bin_Range': bin_range,
                            'Bin_Min': bin_min,
                            'Bin_Max': bin_max,
                            'Base_Percentage': (train_baseline.get(bin_name, 0) * 100),
                            'Actual_Percentage': (actual_dist.get(bin_name, 0) * 100),
                            'Bin_PSI': bin_psi
                        })

    return pd.DataFrame(results)

# SIL

## Queries

#### Alpha - CIC-SIL-Model

##### Test

In [8]:
## This is for the test period of Alpha - CIC sil model - reading the data from ml_model_run_details

sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  )
SELECT
  r.customerId,r.digitalLoanAccountId,prediction Alpha_cic_sil_score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'Alpha - CIC-SIL-Model' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
;
"""
dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

## this data is not expanded. We will have to expand and get the features from the calcFeatures column

The shape of the dataframe is: (100808, 19)


,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,1432731,51016ab9-7f03-40db-90d9-02c39cbcfd18,0.11580512192886502,2025-11-22 05:30:01.372179,2025-11-22 05:30:01.378310,cic_model_sil,v1,SIL Competitor,F,Appliance,ios,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1763769600000,""cic_Personal_Loans_...",Test,2025-11-22 13:29:52,2025-11-22 13:35:41,2025-11
1,1543309,014dc675-a37d-4801-a7b6-73364579c9c1,0.17276124598701692,2025-06-26 09:52:05.562621,2025-06-26 09:52:05.568909,cic_model_sil,v1,SIL Competitor,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1750896000000,""cic_Personal_Loans_...",Test,2025-06-26 17:51:55,2025-06-26 18:00:18,2025-06
2,1744871,2e1eb846-4e02-405f-aa08-01770578e647,0.14526991814317072,2025-06-15 10:15:12.304204,2025-06-15 10:15:12.310663,cic_model_sil,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1749945600000,""cic_Personal_Loans_...",Test,2025-06-15 18:15:04,NaT,2025-06
3,1956578,34377540-8ae5-4a11-b96f-b8f539e30f1b,0.12175700239172996,2025-11-23 05:53:44.064775,2025-11-23 05:53:44.071629,cic_model_sil,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1763856000000,""cic_Personal_Loans_...",Test,2025-11-23 13:53:34,2025-11-23 13:58:57,2025-11
4,2256677,cb0c7b4c-f22a-4200-adf9-9dd397028726,0.06219790405519965,2025-08-27 02:52:54.598201,2025-08-27 02:52:54.604533,cic_model_sil,v1,SIL Competitor,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1756252800000,""cic_Personal_Loans_...",Test,2025-08-27 10:52:44,NaT,2025-08


In [9]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()
df1.head()


Original columns: 19
Expanded columns: 48


,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month,calc_run_date,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_days_since_last_inquiry,calc_cic_cnt_active_contracts,calc_cic_vel_contract_nongranted_cnt_12on24,calc_cic_max_amt_granted_24M,calc_cic_zero_non_granted_ever_flag,calc_cic_tot_active_contracts_util,calc_cic_vel_contract_granted_amt_12on24,calc_cic_zero_granted_ever_flag,calc_digitalLoanAccountId,calc_customerId,calc_crifApplicationId,calc_ScoreRange,calc_ln_loan_level_user_type,calc_flg_zero_non_granted_ever,calc_flg_zero_granted_ever,calc_Personal_Loans_granted_contracts_amt_24M,calc_granted_contracts_cnt_6M,calc_total_overdue_granted_contracts,calc_has_ever_been_overdue,calc_cnt_nongranted_contracts_3M,calc_cnt_active_contracts,calc_max_amt_granted_24M,calc_tot_active_contracts_util,calc_days_since_last_closed,calc_vel_contract_nongranted_cnt_6on12,calc_vel_contract_granted_amt_6on12,calc_vel_contract_closed_amt_3on12
0,1432731,51016ab9-7f03-40db-90d9-02c39cbcfd18,0.11580512192886502,2025-11-22 05:30:01.372179,2025-11-22 05:30:01.378310,cic_model_sil,v1,SIL Competitor,F,Appliance,ios,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1763769600000,""cic_Personal_Loans_...",Test,2025-11-22 13:29:52,2025-11-22 13:35:41,2025-11,1.763770e+12,NaN,229.0,1.0,1.97619,NaN,0.0,0.685734,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1543309,014dc675-a37d-4801-a7b6-73364579c9c1,0.17276124598701692,2025-06-26 09:52:05.562621,2025-06-26 09:52:05.568909,cic_model_sil,v1,SIL Competitor,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1750896000000,""cic_Personal_Loans_...",Test,2025-06-26 17:51:55,2025-06-26 18:00:18,2025-06,1.750896e+12,26630.0,3.0,2.0,2.00000,102000.0,0.0,0.376839,1.076967,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1744871,2e1eb846-4e02-405f-aa08-01770578e647,0.14526991814317072,2025-06-15 10:15:12.304204,2025-06-15 10:15:12.310663,cic_model_sil,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1749945600000,""cic_Personal_Loans_...",Test,2025-06-15 18:15:04,NaT,2025-06,1.749946e+12,90.0,178.0,NaN,1.33600,14399.0,0.0,NaN,0.012423,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1956578,34377540-8ae5-4a11-b96f-b8f539e30f1b,0.12175700239172996,2025-11-23 05:53:44.064775,2025-11-23 05:53:44.071629,cic_model_sil,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1763856000000,""cic_Personal_Loans_...",Test,2025-11-23 13:53:34,2025-11-23 13:58:57,2025-11,1.763856e+12,29662.0,114.0,1.0,1.97619,29350.0,0.0,1.195605,1.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2256677,cb0c7b4c-f22a-4200-adf9-9dd397028726,0.06219790405519965,2025-08-27 02:52:54.598201,2025-08-27 02:52:54.604533,cic_model_sil,v1,SIL Competitor,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1756252800000,""cic_Personal_Loans_...",Test,2025-08-27 10:52:44,NaT,2025-08,1.756253e+12,37636.0,396.0,NaN,NaN,26252.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Train

In [10]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  )
SELECT
  r.customerId,r.digitalLoanAccountId,prediction Alpha_cic_sil_score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'Alpha - CIC-SIL-Model' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  'Train' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by cast(r.start_time as datetime) desc) = 1
;
"""
dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

The shape of the dataframe is: (329187, 19)


,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,1017864,4dcbf449-edf1-4fbd-aee8-e74c43c9a8a8,0.087162,2025-12-01T09:00:51.850388,2025-12-01T09:00:51.850388,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_days_since_last_inquiry"": 58.0, ""cic_cnt...",Train,2024-06-14 17:30:26,2024-06-14 17:39:33,2024-06
1,1100922,d77d6f91-d27c-4035-9caf-5b49d3c9cca5,0.146275,2025-12-01T08:59:45.481422,2025-12-01T08:59:45.481422,Alpha - CIC-SIL-Model,v1,SIL Repeat,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_days_since_last_inquiry"": 18.0, ""cic_cnt...",Train,2024-12-27 11:19:26,2024-12-27 11:22:02,2024-12
2,1170270,a949ec32-55e5-4685-a34a-34a3c0b567cc,0.176308,2025-12-01T09:24:57.655557,2025-12-01T09:24:57.655557,cic_model_sil,v2,SIL-Instore,F,Mall,android,Alpha - CIC-SIL-Model,SIL,NA,"{""ScoreRange"": ""Ai"", ""ln_loan_level_user_type""...",Train,2025-08-16 14:18:58,NaT,2025-08
3,1478828,648bd2c2-590a-47f5-9df2-5e5ed5c7fa84,0.057237,2025-12-01T09:00:08.688974,2025-12-01T09:00:08.688974,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_Personal_Loans_granted_contracts_amt_24M...",Train,2023-05-31 11:49:30,2023-05-31 11:54:32,2023-05
4,1564568,2db871b7-e901-41ec-8cf6-43c4f606b062,0.060710,2025-12-01T09:24:57.742624,2025-12-01T09:24:57.742624,cic_model_sil,v2,SIL Competitor,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""ScoreRange"": ""Ei"", ""ln_loan_level_user_type""...",Train,2025-07-20 12:17:53,2025-07-21 10:25:52,2025-07


In [11]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()
df2.head()


Original columns: 19
Expanded columns: 44


,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month,calc_cic_days_since_last_inquiry,calc_cic_cnt_active_contracts,calc_cic_zero_non_granted_ever_flag,calc_cic_tot_active_contracts_util,calc_cic_zero_granted_ever_flag,calc_cic_vel_contract_nongranted_cnt_12on24,calc_cic_max_amt_granted_24M,calc_ScoreRange,calc_ln_loan_level_user_type,calc_flg_zero_non_granted_ever,calc_flg_zero_granted_ever,calc_Personal_Loans_granted_contracts_amt_24M,calc_total_overdue_granted_contracts,calc_has_ever_been_overdue,calc_cnt_active_contracts,calc_max_amt_granted_24M,calc_tot_active_contracts_util,calc_vel_contract_nongranted_cnt_6on12,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_granted_contracts_cnt_6M,calc_days_since_last_closed,calc_vel_contract_granted_amt_6on12,calc_vel_contract_closed_amt_3on12,calc_cic_vel_contract_granted_amt_12on24,calc_cnt_nongranted_contracts_3M
0,1017864,4dcbf449-edf1-4fbd-aee8-e74c43c9a8a8,0.087162,2025-12-01T09:00:51.850388,2025-12-01T09:00:51.850388,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_days_since_last_inquiry"": 58.0, ""cic_cnt...",Train,2024-06-14 17:30:26,2024-06-14 17:39:33,2024-06,58.0,3.0,0.0,0.447583,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1100922,d77d6f91-d27c-4035-9caf-5b49d3c9cca5,0.146275,2025-12-01T08:59:45.481422,2025-12-01T08:59:45.481422,Alpha - CIC-SIL-Model,v1,SIL Repeat,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_days_since_last_inquiry"": 18.0, ""cic_cnt...",Train,2024-12-27 11:19:26,2024-12-27 11:22:02,2024-12,18.0,2.0,0.0,1.071511,0.0,2.0,45000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1170270,a949ec32-55e5-4685-a34a-34a3c0b567cc,0.176308,2025-12-01T09:24:57.655557,2025-12-01T09:24:57.655557,cic_model_sil,v2,SIL-Instore,F,Mall,android,Alpha - CIC-SIL-Model,SIL,NA,"{""ScoreRange"": ""Ai"", ""ln_loan_level_user_type""...",Train,2025-08-16 14:18:58,NaT,2025-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ai,2_New Applicant,0,0,32370.0,0.0,0.0,2.0,100000.0,0.894991,1.994012,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1478828,648bd2c2-590a-47f5-9df2-5e5ed5c7fa84,0.057237,2025-12-01T09:00:08.688974,2025-12-01T09:00:08.688974,Alpha - CIC-SIL-Model,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""cic_Personal_Loans_granted_contracts_amt_24M...",Train,2023-05-31 11:49:30,2023-05-31 11:54:32,2023-05,396.0,NaN,1.0,NaN,0.0,NaN,18562.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18562.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1564568,2db871b7-e901-41ec-8cf6-43c4f606b062,0.060710,2025-12-01T09:24:57.742624,2025-12-01T09:24:57.742624,cic_model_sil,v2,SIL Competitor,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""ScoreRange"": ""Ei"", ""ln_loan_level_user_type""...",Train,2025-07-20 12:17:53,2025-07-21 10:25:52,2025-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ei,2_New Applicant,0,0,395191.0,0.0,0.0,24.0,75000.0,0.296790,NaN,NaN,4.0,29.0,0.601547,1.826845,NaN,NaN


In [12]:
df1.columns

Index(['customerId', 'digitalLoanAccountId', 'Alpha_cic_sil_score',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
       'new_loan_type', 'gender', 'loan_product_type', 'osType', 'Model_Name',
       'product', 'trenchCategory', 'calcFeatures', 'Data_selection',
       'appln_submit_datetime', 'disbursementDateTime', 'Application_month',
       'calc_run_date', 'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_days_since_last_inquiry', 'calc_cic_cnt_active_contracts',
       'calc_cic_vel_contract_nongranted_cnt_12on24',
       'calc_cic_max_amt_granted_24M', 'calc_cic_zero_non_granted_ever_flag',
       'calc_cic_tot_active_contracts_util',
       'calc_cic_vel_contract_granted_amt_12on24',
       'calc_cic_zero_granted_ever_flag', 'calc_digitalLoanAccountId',
       'calc_customerId', 'calc_crifApplicationId', 'calc_ScoreRange',
       'calc_ln_loan_level_user_type', 'calc_flg_zero_non_granted_ever',
       'calc_flg_zero_granted_ever',
  

In [13]:
df2.columns

Index(['customerId', 'digitalLoanAccountId', 'Alpha_cic_sil_score',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
       'new_loan_type', 'gender', 'loan_product_type', 'osType', 'Model_Name',
       'product', 'trenchCategory', 'calcFeatures', 'Data_selection',
       'appln_submit_datetime', 'disbursementDateTime', 'Application_month',
       'calc_cic_days_since_last_inquiry', 'calc_cic_cnt_active_contracts',
       'calc_cic_zero_non_granted_ever_flag',
       'calc_cic_tot_active_contracts_util', 'calc_cic_zero_granted_ever_flag',
       'calc_cic_vel_contract_nongranted_cnt_12on24',
       'calc_cic_max_amt_granted_24M', 'calc_ScoreRange',
       'calc_ln_loan_level_user_type', 'calc_flg_zero_non_granted_ever',
       'calc_flg_zero_granted_ever',
       'calc_Personal_Loans_granted_contracts_amt_24M',
       'calc_total_overdue_granted_contracts', 'calc_has_ever_been_overdue',
       'calc_cnt_active_contracts', 'calc_max_amt_granted_24M',
       'calc_t

##### concatenate

In [14]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.head()

,customerId,digitalLoanAccountId,Alpha_cic_sil_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month,calc_run_date,calc_cic_Personal_Loans_granted_contracts_amt_24M,calc_cic_days_since_last_inquiry,calc_cic_cnt_active_contracts,calc_cic_vel_contract_nongranted_cnt_12on24,calc_cic_max_amt_granted_24M,calc_cic_zero_non_granted_ever_flag,calc_cic_tot_active_contracts_util,calc_cic_vel_contract_granted_amt_12on24,calc_cic_zero_granted_ever_flag,calc_digitalLoanAccountId,calc_customerId,calc_crifApplicationId,calc_ScoreRange,calc_ln_loan_level_user_type,calc_flg_zero_non_granted_ever,calc_flg_zero_granted_ever,calc_Personal_Loans_granted_contracts_amt_24M,calc_granted_contracts_cnt_6M,calc_total_overdue_granted_contracts,calc_has_ever_been_overdue,calc_cnt_nongranted_contracts_3M,calc_cnt_active_contracts,calc_max_amt_granted_24M,calc_tot_active_contracts_util,calc_days_since_last_closed,calc_vel_contract_nongranted_cnt_6on12,calc_vel_contract_granted_amt_6on12,calc_vel_contract_closed_amt_3on12
0,1432731,51016ab9-7f03-40db-90d9-02c39cbcfd18,0.11580512192886502,2025-11-22 05:30:01.372179,2025-11-22 05:30:01.378310,cic_model_sil,v1,SIL Competitor,F,Appliance,ios,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1763769600000,""cic_Personal_Loans_...",Test,2025-11-22 13:29:52,2025-11-22 13:35:41,2025-11,1.763770e+12,NaN,229.0,1.0,1.97619,NaN,0.0,0.685734,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1543309,014dc675-a37d-4801-a7b6-73364579c9c1,0.17276124598701692,2025-06-26 09:52:05.562621,2025-06-26 09:52:05.568909,cic_model_sil,v1,SIL Competitor,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1750896000000,""cic_Personal_Loans_...",Test,2025-06-26 17:51:55,2025-06-26 18:00:18,2025-06,1.750896e+12,26630.0,3.0,2.0,2.00000,102000.0,0.0,0.376839,1.076967,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1744871,2e1eb846-4e02-405f-aa08-01770578e647,0.14526991814317072,2025-06-15 10:15:12.304204,2025-06-15 10:15:12.310663,cic_model_sil,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1749945600000,""cic_Personal_Loans_...",Test,2025-06-15 18:15:04,NaT,2025-06,1.749946e+12,90.0,178.0,NaN,1.33600,14399.0,0.0,NaN,0.012423,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1956578,34377540-8ae5-4a11-b96f-b8f539e30f1b,0.12175700239172996,2025-11-23 05:53:44.064775,2025-11-23 05:53:44.071629,cic_model_sil,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1763856000000,""cic_Personal_Loans_...",Test,2025-11-23 13:53:34,2025-11-23 13:58:57,2025-11,1.763856e+12,29662.0,114.0,1.0,1.97619,29350.0,0.0,1.195605,1.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2256677,cb0c7b4c-f22a-4200-adf9-9dd397028726,0.06219790405519965,2025-08-27 02:52:54.598201,2025-08-27 02:52:54.604533,cic_model_sil,v1,SIL Competitor,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,NA,"{""run_date"":1756252800000,""cic_Personal_Loans_...",Test,2025-08-27 10:52:44,NaT,2025-08,1.756253e+12,37636.0,396.0,NaN,NaN,26252.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_concat.groupby(['Data_selection','Application_month'])['digitalLoanAccountId'].nunique().reset_index().sort_values(by=['Application_month','Data_selection'])

,Data_selection,Application_month,digitalLoanAccountId
10,Train,2023-01,292
11,Train,2023-02,323
12,Train,2023-03,364
13,Train,2023-04,446
14,Train,2023-05,453
15,Train,2023-06,507
16,Train,2023-07,521
17,Train,2023-08,1148
18,Train,2023-09,1886
19,Train,2023-10,1836


In [16]:
df = df_concat.drop(columns=['calcFeatures']).copy()

In [17]:
df['Alpha_cic_sil_score'] = pd.to_numeric(df['Alpha_cic_sil_score'], errors='coerce')

##### PSI calculation

In [18]:
df = df.copy()

# Define feature list
feature_list = ['Alpha_cic_sil_score',  'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_days_since_last_inquiry', 'calc_cic_cnt_active_contracts',
       'calc_cic_vel_contract_nongranted_cnt_12on24',
       'calc_cic_max_amt_granted_24M', 'calc_cic_zero_non_granted_ever_flag',
       'calc_cic_tot_active_contracts_util',
       'calc_cic_vel_contract_granted_amt_12on24',
       'calc_cic_zero_granted_ever_flag']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

Training period: 2023-01 to 2025-11
Test period: 2025-03 to 2025-12
Feature 'Alpha_cic_sil_score': Created 10 bins
Feature 'calc_cic_Personal_Loans_granted_contracts_amt_24M': Created 10 bins
Feature 'calc_cic_days_since_last_inquiry': Created 5 bins
Feature 'calc_cic_cnt_active_contracts': Using equal distance bins of 5
Feature 'calc_cic_cnt_active_contracts': Created 5 bins
Feature 'calc_cic_vel_contract_nongranted_cnt_12on24': Using equal distance bins of 5
Feature 'calc_cic_vel_contract_nongranted_cnt_12on24': Created 5 bins
Feature 'calc_cic_max_amt_granted_24M': Created 10 bins
Feature 'calc_cic_tot_active_contracts_util': Created 10 bins
Feature 'calc_cic_vel_contract_granted_amt_12on24': Using equal distance bins of 5
Feature 'calc_cic_vel_contract_granted_amt_12on24': Created 5 bins
Unique categories: [1.0, 0.0]
Top categories for feature 'calc_cic_zero_non_granted_ever_flag': [1.0, 0.0]
Unique categories: [0.0, 1.0]
Top categories for feature 'calc_cic_zero_granted_ever_flag'

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ad0bcc26-1b4d-4e41-a2fc-6fad3e60b97a>

#### Alpha - IncomeEstimationModel

##### Test

In [19]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName = 'Alpha  - IncomeEstimationModel')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction Alpha_Income_Estimated_score,start_time,end_time,modelDisplayName,modelVersionId,
  loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
  'Alpha  - IncomeEstimationModel' Model_Name,
  'SIL' as product,
  'NA' trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  JSON_VALUE(calcFeature, "$.inc_alpha_cic_credit_avg_credit_limit") AS inc_alpha_cic_credit_avg_credit_limit,
  JSON_VALUE(calcFeature, "$.inc_alpha_cic_max_active_contracts_amt") AS inc_alpha_cic_max_active_contracts_amt,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_age") AS inc_alpha_ln_age,
  JSON_VALUE(calcFeature, "$.inc_alpha_doc_type_rolled") AS inc_alpha_doc_type_rolled,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_brand") AS inc_alpha_ln_brand,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_city") AS inc_alpha_ln_city,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_cnt_dependents") AS inc_alpha_ln_cnt_dependents,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_education_level") AS inc_alpha_ln_education_level,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_employment_type_new") AS inc_alpha_ln_employment_type_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_gender") AS inc_alpha_ln_gender,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_industry_new") AS inc_alpha_ln_industry_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_loan_prod_type") AS inc_alpha_ln_loan_prod_type,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_marital_status_new") AS inc_alpha_ln_marital_status_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_nature_of_work_new") AS inc_alpha_ln_nature_of_work_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_osversion_bin") AS inc_alpha_ln_osversion_bin,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_purpose") AS inc_alpha_ln_purpose,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_source_of_funds_new") AS inc_alpha_ln_source_of_funds_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_encoded_company_name_grouped") AS inc_alpha_encoded_company_name_grouped,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by r.start_time desc) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,Alpha_Income_Estimated_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,inc_alpha_cic_credit_avg_credit_limit,inc_alpha_cic_max_active_contracts_amt,inc_alpha_ln_age,inc_alpha_doc_type_rolled,inc_alpha_ln_brand,inc_alpha_ln_city,inc_alpha_ln_cnt_dependents,inc_alpha_ln_education_level,inc_alpha_ln_employment_type_new,inc_alpha_ln_gender,inc_alpha_ln_industry_new,inc_alpha_ln_loan_prod_type,inc_alpha_ln_marital_status_new,inc_alpha_ln_nature_of_work_new,inc_alpha_ln_osversion_bin,inc_alpha_ln_purpose,inc_alpha_ln_source_of_funds_new,inc_alpha_encoded_company_name_grouped,appln_submit_datetime,disbursementDateTime,Application_month


In [20]:
# Expand the calcFeatures column
df1 = dfd.copy()

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {df1.shape[1]}")
df1.head()

Original columns: 37
Expanded columns: 37


,customerId,digitalLoanAccountId,Alpha_Income_Estimated_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,inc_alpha_cic_credit_avg_credit_limit,inc_alpha_cic_max_active_contracts_amt,inc_alpha_ln_age,inc_alpha_doc_type_rolled,inc_alpha_ln_brand,inc_alpha_ln_city,inc_alpha_ln_cnt_dependents,inc_alpha_ln_education_level,inc_alpha_ln_employment_type_new,inc_alpha_ln_gender,inc_alpha_ln_industry_new,inc_alpha_ln_loan_prod_type,inc_alpha_ln_marital_status_new,inc_alpha_ln_nature_of_work_new,inc_alpha_ln_osversion_bin,inc_alpha_ln_purpose,inc_alpha_ln_source_of_funds_new,inc_alpha_encoded_company_name_grouped,appln_submit_datetime,disbursementDateTime,Application_month


##### Train

In [21]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName = 'Alpha  - IncomeEstimationModel')
SELECT
  r.customerId,r.digitalLoanAccountId,prediction Alpha_Income_Estimated_score,start_time,end_time,modelDisplayName,modelVersionId,
  loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
  'Alpha  - IncomeEstimationModel' Model_Name,
  'SIL' as product,
  'NA' trenchCategory,
  r.calcFeature calcFeatures,
  'Train' Data_selection,
  JSON_VALUE(calcFeature, "$.inc_alpha_cic_credit_avg_credit_limit") AS inc_alpha_cic_credit_avg_credit_limit,
  JSON_VALUE(calcFeature, "$.inc_alpha_cic_max_active_contracts_amt") AS inc_alpha_cic_max_active_contracts_amt,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_age") AS inc_alpha_ln_age,
  JSON_VALUE(calcFeature, "$.inc_alpha_doc_type_rolled") AS inc_alpha_doc_type_rolled,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_brand") AS inc_alpha_ln_brand,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_city") AS inc_alpha_ln_city,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_cnt_dependents") AS inc_alpha_ln_cnt_dependents,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_education_level") AS inc_alpha_ln_education_level,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_employment_type_new") AS inc_alpha_ln_employment_type_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_gender") AS inc_alpha_ln_gender,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_industry_new") AS inc_alpha_ln_industry_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_loan_prod_type") AS inc_alpha_ln_loan_prod_type,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_marital_status_new") AS inc_alpha_ln_marital_status_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_nature_of_work_new") AS inc_alpha_ln_nature_of_work_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_osversion_bin") AS inc_alpha_ln_osversion_bin,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_purpose") AS inc_alpha_ln_purpose,
  JSON_VALUE(calcFeature, "$.inc_alpha_ln_source_of_funds_new") AS inc_alpha_ln_source_of_funds_new,
  JSON_VALUE(calcFeature, "$.inc_alpha_encoded_company_name_grouped") AS inc_alpha_encoded_company_name_grouped,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId order by cast(r.start_time as datetime) desc) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()


The shape of the dataframe is: (349207, 37)


,customerId,digitalLoanAccountId,Alpha_Income_Estimated_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,inc_alpha_cic_credit_avg_credit_limit,inc_alpha_cic_max_active_contracts_amt,inc_alpha_ln_age,inc_alpha_doc_type_rolled,inc_alpha_ln_brand,inc_alpha_ln_city,inc_alpha_ln_cnt_dependents,inc_alpha_ln_education_level,inc_alpha_ln_employment_type_new,inc_alpha_ln_gender,inc_alpha_ln_industry_new,inc_alpha_ln_loan_prod_type,inc_alpha_ln_marital_status_new,inc_alpha_ln_nature_of_work_new,inc_alpha_ln_osversion_bin,inc_alpha_ln_purpose,inc_alpha_ln_source_of_funds_new,inc_alpha_encoded_company_name_grouped,appln_submit_datetime,disbursementDateTime,Application_month
0,1000020,27c931c2-8d0a-4be6-b121-2c063730cf4b,9147.381480,2025-12-01T09:08:51.348119,2025-12-01T09:08:51.348119,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Appliance,ios,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",Train,0.0,0.0,37.0,UMID Card,Apple,CITY OF MANILA,No dependents,College Graduate,Employed - Private Employee,F,Banking,Mall,Single,Bank Employee,3-ComfortSeeker,Televisions,Salary,0.0,2024-04-04 12:58:20,NaT,2024-04
1,1017157,8ff5ab43-1954-4fd4-bbf1-8776e8a753c0,25659.283066,2025-12-01T09:09:47.593162,2025-12-01T09:09:47.593162,Alpha - IncomeEstimationModel,v1,SIL ZERO,F,Appliance,android,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",Train,0.0,28008.0,28.0,ID Card,Redmi,BALIUAG,No dependents,Post-Graduate/Doctorate Degree Holder,Employed - Govt. Employee/Govt. Official,F,Govt. Institution,Mall,Married,Govt Employee,2-Voyager,Air conditioners,Salary,10142518.046121446,2024-09-28 17:55:31,2024-09-28 18:00:11,2024-09
2,1058512,4f730007-0767-4b65-9fff-b5f8fcf3a3e8,48766.493815,2025-12-01T09:09:25.982910,2025-12-01T09:09:25.982910,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Mobile,ios,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",Train,0.0,0.0,27.0,Passport,Apple,RODRIGUEZ (MONTALBAN),1,College Graduate,Employed - Govt. Employee/Govt. Official,F,Business Process Outsourcing/Tele Marketing,Mobile,Single,Call Center Agent/Tele Marketer,5-Pioneer,Mobile Phones,Salary,32086.7246045263,2024-12-21 12:06:50,NaT,2024-12
3,1183491,95b10a2c-39cf-400b-9a48-06795e3ce8e6,8637.399182,2025-12-01T09:08:37.060017,2025-12-01T09:08:37.060017,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Appliance,android,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",Train,0.0,0.0,36.0,UMID Card,realme,BINANGONAN,2,College Graduate,Employed - Govt. Employee/Govt. Official,F,Hospital and Medical Services,Mall,Single,Other Professional services,2-Voyager,Refrigerators,Salary,0.0,2023-11-26 16:43:50,2023-11-26 16:51:18,2023-11
4,1262166,d61d6acd-0071-49d8-8a6b-4bab50418f8d,35003.957134,2025-12-01T09:09:07.666995,2025-12-01T09:09:07.666995,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Appliance,android,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",Train,0.0,35000.0,26.0,ID Card,Itel,NAGA CITY,No dependents,College Graduate,Employed - Govt. Employee/Govt. Official,F,Business Process Outsourcing/Tele Marketing,Appliance,Single,Call Center Agent/Tele Marketer,2-Voyager,Air conditioners,Salary,65326.77572616716,2024-05-25 14:39:59,2024-05-25 14:42:51,2024-05


In [22]:
df2 = dfd.copy()

In [23]:
df1.columns

Index(['customerId', 'digitalLoanAccountId', 'Alpha_Income_Estimated_score',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
       'new_loan_type', 'gender', 'loan_product_type', 'osType', 'Model_Name',
       'product', 'trenchCategory', 'calcFeatures', 'Data_selection',
       'inc_alpha_cic_credit_avg_credit_limit',
       'inc_alpha_cic_max_active_contracts_amt', 'inc_alpha_ln_age',
       'inc_alpha_doc_type_rolled', 'inc_alpha_ln_brand', 'inc_alpha_ln_city',
       'inc_alpha_ln_cnt_dependents', 'inc_alpha_ln_education_level',
       'inc_alpha_ln_employment_type_new', 'inc_alpha_ln_gender',
       'inc_alpha_ln_industry_new', 'inc_alpha_ln_loan_prod_type',
       'inc_alpha_ln_marital_status_new', 'inc_alpha_ln_nature_of_work_new',
       'inc_alpha_ln_osversion_bin', 'inc_alpha_ln_purpose',
       'inc_alpha_ln_source_of_funds_new',
       'inc_alpha_encoded_company_name_grouped', 'appln_submit_datetime',
       'disbursementDateTime', 'Application_month

In [24]:
df2.columns

Index(['customerId', 'digitalLoanAccountId', 'Alpha_Income_Estimated_score',
       'start_time', 'end_time', 'modelDisplayName', 'modelVersionId',
       'new_loan_type', 'gender', 'loan_product_type', 'osType', 'Model_Name',
       'product', 'trenchCategory', 'calcFeatures', 'Data_selection',
       'inc_alpha_cic_credit_avg_credit_limit',
       'inc_alpha_cic_max_active_contracts_amt', 'inc_alpha_ln_age',
       'inc_alpha_doc_type_rolled', 'inc_alpha_ln_brand', 'inc_alpha_ln_city',
       'inc_alpha_ln_cnt_dependents', 'inc_alpha_ln_education_level',
       'inc_alpha_ln_employment_type_new', 'inc_alpha_ln_gender',
       'inc_alpha_ln_industry_new', 'inc_alpha_ln_loan_prod_type',
       'inc_alpha_ln_marital_status_new', 'inc_alpha_ln_nature_of_work_new',
       'inc_alpha_ln_osversion_bin', 'inc_alpha_ln_purpose',
       'inc_alpha_ln_source_of_funds_new',
       'inc_alpha_encoded_company_name_grouped', 'appln_submit_datetime',
       'disbursementDateTime', 'Application_month

##### concatenate

In [25]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.head()

,customerId,digitalLoanAccountId,Alpha_Income_Estimated_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,inc_alpha_cic_credit_avg_credit_limit,inc_alpha_cic_max_active_contracts_amt,inc_alpha_ln_age,inc_alpha_doc_type_rolled,inc_alpha_ln_brand,inc_alpha_ln_city,inc_alpha_ln_cnt_dependents,inc_alpha_ln_education_level,inc_alpha_ln_employment_type_new,inc_alpha_ln_gender,inc_alpha_ln_industry_new,inc_alpha_ln_loan_prod_type,inc_alpha_ln_marital_status_new,inc_alpha_ln_nature_of_work_new,inc_alpha_ln_osversion_bin,inc_alpha_ln_purpose,inc_alpha_ln_source_of_funds_new,inc_alpha_encoded_company_name_grouped,appln_submit_datetime,disbursementDateTime,Application_month
0,1000020,27c931c2-8d0a-4be6-b121-2c063730cf4b,9147.381480,2025-12-01T09:08:51.348119,2025-12-01T09:08:51.348119,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Appliance,ios,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",Train,0.0,0.0,37.0,UMID Card,Apple,CITY OF MANILA,No dependents,College Graduate,Employed - Private Employee,F,Banking,Mall,Single,Bank Employee,3-ComfortSeeker,Televisions,Salary,0.0,2024-04-04 12:58:20,NaT,2024-04
1,1017157,8ff5ab43-1954-4fd4-bbf1-8776e8a753c0,25659.283066,2025-12-01T09:09:47.593162,2025-12-01T09:09:47.593162,Alpha - IncomeEstimationModel,v1,SIL ZERO,F,Appliance,android,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",Train,0.0,28008.0,28.0,ID Card,Redmi,BALIUAG,No dependents,Post-Graduate/Doctorate Degree Holder,Employed - Govt. Employee/Govt. Official,F,Govt. Institution,Mall,Married,Govt Employee,2-Voyager,Air conditioners,Salary,10142518.046121446,2024-09-28 17:55:31,2024-09-28 18:00:11,2024-09
2,1058512,4f730007-0767-4b65-9fff-b5f8fcf3a3e8,48766.493815,2025-12-01T09:09:25.982910,2025-12-01T09:09:25.982910,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Mobile,ios,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",Train,0.0,0.0,27.0,Passport,Apple,RODRIGUEZ (MONTALBAN),1,College Graduate,Employed - Govt. Employee/Govt. Official,F,Business Process Outsourcing/Tele Marketing,Mobile,Single,Call Center Agent/Tele Marketer,5-Pioneer,Mobile Phones,Salary,32086.7246045263,2024-12-21 12:06:50,NaT,2024-12
3,1183491,95b10a2c-39cf-400b-9a48-06795e3ce8e6,8637.399182,2025-12-01T09:08:37.060017,2025-12-01T09:08:37.060017,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Appliance,android,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",Train,0.0,0.0,36.0,UMID Card,realme,BINANGONAN,2,College Graduate,Employed - Govt. Employee/Govt. Official,F,Hospital and Medical Services,Mall,Single,Other Professional services,2-Voyager,Refrigerators,Salary,0.0,2023-11-26 16:43:50,2023-11-26 16:51:18,2023-11
4,1262166,d61d6acd-0071-49d8-8a6b-4bab50418f8d,35003.957134,2025-12-01T09:09:07.666995,2025-12-01T09:09:07.666995,Alpha - IncomeEstimationModel,v1,SIL-Instore,F,Appliance,android,Alpha - IncomeEstimationModel,SIL,NA,"{""inc_alpha_cic_credit_avg_credit_limit"": 0.0,...",Train,0.0,35000.0,26.0,ID Card,Itel,NAGA CITY,No dependents,College Graduate,Employed - Govt. Employee/Govt. Official,F,Business Process Outsourcing/Tele Marketing,Appliance,Single,Call Center Agent/Tele Marketer,2-Voyager,Air conditioners,Salary,65326.77572616716,2024-05-25 14:39:59,2024-05-25 14:42:51,2024-05


In [26]:
df_concat.groupby(['Data_selection','Application_month'])['digitalLoanAccountId'].nunique().reset_index().sort_values(by=['Application_month','Data_selection'])

,Data_selection,Application_month,digitalLoanAccountId
0,Train,2023-01,1331
1,Train,2023-02,981
2,Train,2023-03,1161
3,Train,2023-04,1324
4,Train,2023-05,1391
5,Train,2023-06,1476
6,Train,2023-07,2309
7,Train,2023-08,3918
8,Train,2023-09,5598
9,Train,2023-10,6461


##### PSI calculation

In [27]:
df = df_concat.drop(columns=['calcFeatures']).copy()

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349207 entries, 0 to 349206
Data columns (total 36 columns):
 #   Column                                  Non-Null Count   Dtype         
---  ------                                  --------------   -----         
 0   customerId                              349207 non-null  Int64         
 1   digitalLoanAccountId                    349207 non-null  object        
 2   Alpha_Income_Estimated_score            349207 non-null  float64       
 3   start_time                              349207 non-null  object        
 4   end_time                                349207 non-null  object        
 5   modelDisplayName                        349207 non-null  object        
 6   modelVersionId                          349207 non-null  object        
 7   new_loan_type                           349207 non-null  object        
 8   gender                                  349207 non-null  object        
 9   loan_product_type                    

In [29]:
df['Alpha_Income_Estimated_score'] = pd.to_numeric(df['Alpha_Income_Estimated_score'], errors='coerce')

In [31]:
df = df.copy()

# Define feature list
feature_list = ['Alpha_Income_Estimated_score',   'inc_alpha_cic_credit_avg_credit_limit',
       'inc_alpha_cic_max_active_contracts_amt', 'inc_alpha_ln_age',
       'inc_alpha_doc_type_rolled', 'inc_alpha_ln_brand', 'inc_alpha_ln_city',
       'inc_alpha_ln_cnt_dependents', 'inc_alpha_ln_education_level',
       'inc_alpha_ln_employment_type_new', 'inc_alpha_ln_gender',
       'inc_alpha_ln_industry_new', 'inc_alpha_ln_loan_prod_type',
       'inc_alpha_ln_marital_status_new', 'inc_alpha_ln_nature_of_work_new',
       'inc_alpha_ln_osversion_bin', 'inc_alpha_ln_purpose',
       'inc_alpha_ln_source_of_funds_new',
       'inc_alpha_encoded_company_name_grouped',]

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results.head()
# psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                            'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
#                            'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                            'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_income_estimation_model_psi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

Training period: 2023-01 to 2025-11
Test period: nan to nan
Feature 'Alpha_Income_Estimated_score': Created 10 bins
Unique categories: ['0.0', '20000.0', '25000.0', '30000.0', '15000.0', '40000.0', '50000.0', '10000.0', '35000.0', '5000.0', '45000.0', '100000.0', '60000.0', '65000.0', '55000.0', '70000.0', '75000.0', '80000.0', '2500.0', '90000.0', '9000.0', '8000.0', '85000.0', '18000.0', '22000.0', '200000.0', '24000.0', '105000.0', '95000.0', '23000.0', '120000.0', '13000.0', '125000.0', '14000.0', '1000.0', '500000.0', '150000.0', '19000.0', '36000.0', '16000.0', '39000.0', '26000.0', '130000.0', '12000.0', '27000.0', '218000.0', '17000.0', '28000.0', '135000.0', '32000.0', '17500.0', '115000.0', '110000.0', '11000.0', '140000.0', '102000.0', '21000.0', '165000.0', '31000.0', '27500.0', '29000.0', '46000.0', '33000.0', '22500.0', '38000.0', '37500.0', '52000.0', '54000.0', '37000.0', '48000.0', '6000.0', '190000.0', '66000.0', '2000.0', '32500.0', '9500.0', '155000.0', '34000.0', '

,modelDisplayName,Model_Name,modelVersionId,trenchCategory


#### Alpha Sil Stack Model

##### Test

In [32]:
sq = """ WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Alpha - StackingModel', 'alpha_stack_model_sil')
  )
SELECT distinct
 r.customerId,r.digitalLoanAccountId,prediction Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
    case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'SIL Alpha - StackingModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over(partition by r.digitalLoanAccountId order by r.start_time desc)=1
  ;"""
dfd = client.query(sq).to_dataframe()
dfd = dfd.drop_duplicates(keep='first')
dfd.head()

,customerId,digitalLoanAccountId,Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,3347055,0b04b110-4c7d-48e4-825c-d00e1736ab52,0.017555527319890005,2025-03-29 06:58:37.388070,2025-03-29 06:58:37.391479,alpha_stack_model_sil,v1,SIL-Instore,F,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.21498...",Test,2025-03-29 14:58:28,2025-03-29 15:06:35,2025-03
1,3348528,999b153a-a0f9-4a5c-80ae-d07d946f051c,0.010246952080656055,2025-03-30 03:08:06.093068,2025-03-30 03:08:06.096661,alpha_stack_model_sil,v1,SIL-Instore,M,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.12666...",Test,2025-03-30 11:07:59,2025-03-30 11:13:29,2025-03
2,3347739,5f797781-8a3e-4fd4-ac21-f08d72ff9cb1,0.010459303363630683,2025-03-29 10:53:15.658539,2025-03-29 10:53:15.661954,alpha_stack_model_sil,v1,SIL-Instore,F,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.13664...",Test,2025-03-29 18:53:05,2025-03-29 19:03:30,2025-03
3,3346509,a98b415c-0804-4994-b9c6-8cb026ec6fbb,0.012115481431788264,2025-03-29 03:09:44.755942,2025-03-29 03:09:44.759961,alpha_stack_model_sil,v1,SIL-Instore,M,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.15402...",Test,2025-03-29 11:09:36,2025-03-29 11:18:13,2025-03
4,3351859,ae1568d5-706a-44a6-936e-7e45515ea209,0.010122631917621657,2025-03-31 12:01:07.794300,2025-03-31 12:01:07.797944,alpha_stack_model_sil,v1,SIL-Instore,F,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.11947...",Test,2025-03-31 20:00:57,2025-03-31 20:06:38,2025-03


In [33]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


Original columns: 19
Expanded columns: 33


In [40]:
df1.head()

,customerId,digitalLoanAccountId,Sil_Alpha_Stack_score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month,calc_sb_demo_score,calc_s_cic_score,calc_s_credo_score,calc_s_apps_score,calc_apps_score,calc_sa_cic_score,calc_trench_category,calc_ln_os_type,calc_is_android,calc_sb_demo_score_norm,calc_apps_score_norm,calc_s_credo_score_norm,calc_sa_cic_score_norm,calc_sa_stack_score
0,3347055,0b04b110-4c7d-48e4-825c-d00e1736ab52,0.017555527319890005,2025-03-29 06:58:37.388070,2025-03-29 06:58:37.391479,alpha_stack_model_sil,v1,SIL-Instore,F,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.21498...",Test,2025-03-29 14:58:28,2025-03-29 15:06:35,2025-03,NaN,0.214982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3348528,999b153a-a0f9-4a5c-80ae-d07d946f051c,0.010246952080656055,2025-03-30 03:08:06.093068,2025-03-30 03:08:06.096661,alpha_stack_model_sil,v1,SIL-Instore,M,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.12666...",Test,2025-03-30 11:07:59,2025-03-30 11:13:29,2025-03,NaN,0.126668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3347739,5f797781-8a3e-4fd4-ac21-f08d72ff9cb1,0.010459303363630683,2025-03-29 10:53:15.658539,2025-03-29 10:53:15.661954,alpha_stack_model_sil,v1,SIL-Instore,F,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.13664...",Test,2025-03-29 18:53:05,2025-03-29 19:03:30,2025-03,NaN,0.136644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3346509,a98b415c-0804-4994-b9c6-8cb026ec6fbb,0.012115481431788264,2025-03-29 03:09:44.755942,2025-03-29 03:09:44.759961,alpha_stack_model_sil,v1,SIL-Instore,M,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.15402...",Test,2025-03-29 11:09:36,2025-03-29 11:18:13,2025-03,NaN,0.154030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3351859,ae1568d5-706a-44a6-936e-7e45515ea209,0.010122631917621657,2025-03-31 12:01:07.794300,2025-03-31 12:01:07.797944,alpha_stack_model_sil,v1,SIL-Instore,F,Appliance,android,SIL Alpha - StackingModel,SIL,NA,"{""sb_demo_score"": null, ""s_cic_score"": 0.11947...",Test,2025-03-31 20:00:57,2025-03-31 20:06:38,2025-03,NaN,0.119474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df1['modelVersionId'].value_counts()

modelVersionId
v1    98911
v2     1907
Name: count, dtype: int64

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['Sil_Alpha_Stack_score',   'calc_sb_demo_score',
       'calc_s_cic_score', 'calc_s_credo_score', 'calc_s_apps_score']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# Calculate bin-level PSI
bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                                   'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
                                    'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                                    'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_sil_stack_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete



#### Beta Sil App Score

##### Test

In [ ]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
  )
SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'SIL Beta - AppsScoreModel' Model_Name,
 'SIL' as product,
  'NA' trenchCategory,
  'Test' Data_selection,
  safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64) as sil_beta_app_score,
 calcFeature calcFeatures,
    coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by start_time desc) = 1
;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train

In [ ]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
  )
SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'SIL Beta - AppsScoreModel' Model_Name,
 'SIL' as product,
  'NA' trenchCategory,
  'Train' Data_selection,
  coalesce(prediction, safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64)) as sil_beta_app_score,
 calcFeature calcFeatures,
    IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()

In [ ]:
df1.columns

In [ ]:
df2.columns

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()

In [ ]:
df['sil_beta_app_score'] = pd.to_numeric(df['sil_beta_app_score'], errors='coerce')

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['sil_beta_app_score',
       'calc_app_cnt_rated_for_3plus_ever', 'calc_app_cnt_education_ever',
       'calc_app_cnt_business_ever', 'calc_app_cnt_music_and_audio_ever',
       'calc_app_cnt_travel_and_local_ever', 'calc_app_cnt_finance_7d',
       'calc_app_cnt_absence_tag_30d', 'calc_app_cnt_competitors_30d',
       'calc_app_cnt_absence_tag_90d',    'calc_app_cnt_finance_90d', 'calc_app_cnt_competitors_90d',
       'calc_app_cnt_payday_90d',      'calc_app_median_time_bw_installed_mins_30d',
       'calc_app_first_competitors_install_to_apply_days',
       'calc_app_first_payday_install_to_apply_days',
       'calc_app_vel_finance_30_over_365']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_appscore_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_appscore_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

#### Beta SIL Demo Score

##### Test

In [ ]:
sq = """ WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in  ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  )
SELECT
  r.customerId,  r.digitalLoanAccountId,start_time, prediction sil_beta_demo_score, modelDisplayName,modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
      case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
  case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'SIL Beta - DemoScoreModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature_cleaned calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code

qualify row_number() over(partition by r.customerId, r.digitalLoanAccountId order by start_time desc) = 1;"""

dfd = client.query(sq).to_dataframe()
dfd.head()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train

In [ ]:
sq = """ WITH cleaned AS (
  SELECT
  customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature_cleaned
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in  ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  )
SELECT
  r.customerId,  r.digitalLoanAccountId,start_time, prediction sil_beta_demo_score, modelDisplayName,modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
      case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
  case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'SIL Beta - DemoScoreModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature_cleaned calcFeatures,
  'Train' Data_selection,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code

qualify row_number() over(partition by r.customerId, r.digitalLoanAccountId order by cast(start_time as datetime) desc) = 1;"""

dfd = client.query(sq).to_dataframe()
dfd.head()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()


In [ ]:
df1.columns

In [ ]:
df2.columns

##### Concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()

In [ ]:
df['sil_beta_demo_score'] = pd.to_numeric(df['sil_beta_demo_score'], errors='coerce')

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['sil_beta_demo_score',
       'calc_beta_de_ln_vas_opted_flag',
       'calc_beta_de_ln_doc_type_rolled', 'calc_beta_de_ln_marital_status',
       'calc_beta_de_ln_age_bin', 'calc_beta_de_ln_province_bin',
       'calc_beta_de_ln_ref2_type', 'calc_beta_de_ln_education_level',
       'calc_beta_de_ln_ref1_type', 'calc_beta_de_ln_industry_new_bin',
       'calc_beta_de_ln_appln_day_of_week',
       'calc_beta_de_onb_name_email_match_score',
       'calc_beta_de_ln_employment_type_new_bin', 'calc_beta_de_ln_telconame',
       'calc_beta_de_time_bw_onb_loan_appln_mins',
       'calc_beta_de_ln_source_of_funds_new_bin', 'calc_beta_de_ln_brand_bin',
       'calc_beta_de_ln_email_primary_domain']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_demo_score_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_demo_score_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

#### Beta SIL STACK Score Model

##### Test

In [ ]:
sq = """ WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  )
SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'SIL_Beta - StackScoreModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  prediction sil_beta_stack_score,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId, r.digitalLoanAccountId order by start_time desc) = 1
;
"""
dfd= client.query(sq).to_dataframe()
dfd.head()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train

In [ ]:
sq = """ WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM  prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  )
SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,
 loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'SIL_Beta - StackScoreModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  prediction sil_beta_stack_score,
  'Train' Data_selection,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId, r.digitalLoanAccountId order by cast(start_time as datetime) desc) = 1
;
"""
dfd= client.query(sq).to_dataframe()
dfd.head()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()


In [ ]:
df1.columns

In [ ]:
df2.columns

In [ ]:
df2.rename(columns={'calc_apps_score':'calc_s_apps_score', 'calc_credo_gen_score':'calc_s_credo_score', 'calc_beta_demo_score':'calc_sb_demo_score'}, inplace = True)

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()

In [ ]:
df['sil_beta_stack_score'] = pd.to_numeric(df['sil_beta_stack_score'], errors='coerce')

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['sil_beta_stack_score',
        'calc_s_apps_score', 'calc_s_credo_score', 'calc_sb_demo_score']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_stack_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_stack_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

#### Beta - IncomeEstimationModel

##### Test

In [ ]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in  ('Beta - IncomeEstimationModel', 'beta_income_model')
  )
SELECT
  r.customerId,r.digitalLoanAccountId,prediction sil_beta_income_estimation_score,start_time,end_time,modelDisplayName,modelVersionId,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_loan_type") AS inc_beta_ln_loan_type,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_education_level") AS inc_beta_ln_education_level,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_employment_type_new") AS inc_beta_ln_employment_type_new,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_industry_new") AS inc_beta_ln_industry_new,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_age") AS inc_beta_ln_age,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_brand") AS inc_beta_ln_brand,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_city") AS inc_beta_ln_city,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_purpose") AS inc_beta_ln_purpose,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_osversion_bin") AS inc_beta_ln_osversion_bin,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_postal_code") AS inc_beta_ln_postal_code,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_gender") AS inc_beta_ln_gender,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_doc_type_rolled") AS inc_beta_ln_doc_type_rolled,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_cnt_dependents") AS inc_beta_ln_cnt_dependents,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_source_of_funds_new") AS inc_beta_ln_source_of_funds_new,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_marital_status_new") AS inc_beta_ln_marital_status_new,
  JSON_VALUE(calcFeature, "$.inc_beta_encoded_company_name_grouped") AS inc_beta_encoded_company_name_grouped,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'SIL Beta - IncomeEstimationModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId, r.digitalLoanAccountId order by start_time desc) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

In [ ]:
df1 = dfd.copy()

##### Train

In [ ]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,

    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in  ('Beta - IncomeEstimationModel', 'beta_income_model')
  )
SELECT
  r.customerId,r.digitalLoanAccountId,prediction sil_beta_income_estimation_score,start_time,end_time,modelDisplayName,modelVersionId,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_loan_type") AS inc_beta_ln_loan_type,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_education_level") AS inc_beta_ln_education_level,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_employment_type_new") AS inc_beta_ln_employment_type_new,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_industry_new") AS inc_beta_ln_industry_new,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_age") AS inc_beta_ln_age,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_brand") AS inc_beta_ln_brand,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_city") AS inc_beta_ln_city,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_purpose") AS inc_beta_ln_purpose,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_osversion_bin") AS inc_beta_ln_osversion_bin,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_postal_code") AS inc_beta_ln_postal_code,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_gender") AS inc_beta_ln_gender,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_doc_type_rolled") AS inc_beta_ln_doc_type_rolled,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_cnt_dependents") AS inc_beta_ln_cnt_dependents,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_source_of_funds_new") AS inc_beta_ln_source_of_funds_new,
  JSON_VALUE(calcFeature, "$.inc_beta_ln_marital_status_new") AS inc_beta_ln_marital_status_new,
  JSON_VALUE(calcFeature, "$.inc_beta_encoded_company_name_grouped") AS inc_beta_encoded_company_name_grouped,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
 'SIL Beta - IncomeEstimationModel' Model_Name,
 'SIL' as product,
 'NA' trenchCategory,
  r.calcFeature calcFeatures,
  'Train' Data_selection,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId, r.digitalLoanAccountId order by cast(start_time as datetime) desc) = 1
  ;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

In [ ]:
df2 = dfd.copy()

In [ ]:
df1.columns

In [ ]:
df2.columns

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()

In [ ]:
df['sil_beta_income_estimation_score'] = pd.to_numeric(df['sil_beta_income_estimation_score'], errors='coerce')

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['sil_beta_income_estimation_score',  'inc_beta_ln_loan_type',
       'inc_beta_ln_education_level', 'inc_beta_ln_employment_type_new',
       'inc_beta_ln_industry_new', 'inc_beta_ln_age', 'inc_beta_ln_brand',
       'inc_beta_ln_city', 'inc_beta_ln_purpose', 'inc_beta_ln_osversion_bin',
       'inc_beta_ln_postal_code', 'inc_beta_ln_gender',
       'inc_beta_ln_doc_type_rolled', 'inc_beta_ln_cnt_dependents',
       'inc_beta_ln_source_of_funds_new', 'inc_beta_ln_marital_status_new',
       'inc_beta_encoded_company_name_grouped',]

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

# Cash

#### Alpha-Cash-CIC-Model (All Trench)

##### Trench 1

##### Test

In [ ]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Alpha-Cash-CIC-Model_All_Trench' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 1'
""" 

dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train

In [ ]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Alpha-Cash-CIC-Model_All_Trench' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 1'
;
"""
dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()


In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()

In [ ]:
df1.columns

In [ ]:
df2.columns

In [ ]:
df2.rename(columns = {
    'calc_max_age_all_contracts_snapshot':'calc_cic_max_age_all_contracts_snapshot',
    'calc_ratio_overdue_contracts_to_granted_contracts':'calc_cic_ratio_overdue_contracts_to_granted_contracts',
    'calc_ScoreRange':'calc_cic_ScoreRange',
    'calc_ln_loan_level_user_type':'calc_cic_ln_loan_level_user_type', 
    'calc_has_ever_been_overdue':'calc_cic_has_ever_been_overdue',
    'calc_latest_granted_contract_overdue_flag':'calc_cic_latest_granted_contract_overdue_flag',
    'calc_ratio_closed_over_new_granted_cnt_24M':'calc_cic_ratio_closed_over_new_granted_cnt_24M',
    'calc_ratio_risky_contracts_to_granted_contracts':'calc_cic_ratio_risky_contracts_to_granted_contracts',
    'calc_Short_and_Term_Loans_granted_contracts_cnt_24M':'calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M',
    'calc_flg_zero_non_granted_ever':'calc_cic_flg_zero_non_granted_ever', 
    'calc_CreditAvgCreditLimit':'calc_cic_CreditAvgCreditLimit',
    'calc_flg_zero_granted_ever':'calc_cic_flg_zero_granted_ever', 
    'calc_ca_cic_score':'calc_cic_ca_cic_score',
    'calc_Personal_Loans_granted_contracts_amt_24M':'calc_cic_Personal_Loans_granted_contracts_amt_24M'
    
}, inplace = True)

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()

In [ ]:
df['aCicScore'] = pd.to_numeric(df['aCicScore'], errors='coerce')

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['aCicScore',
     'calc_cic_max_age_all_contracts_snapshot',
       'calc_cic_ratio_overdue_contracts_to_granted_contracts',
       'calc_cic_ScoreRange', 'calc_cic_ln_loan_level_user_type',
       'calc_cic_has_ever_been_overdue',
       'calc_cic_latest_granted_contract_overdue_flag',
       'calc_cic_ratio_closed_over_new_granted_cnt_24M',
       'calc_cic_ratio_risky_contracts_to_granted_contracts',
       'calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M',
       'calc_cic_flg_zero_non_granted_ever',
       'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_CreditAvgCreditLimit', 'calc_cic_flg_zero_granted_ever'
]
# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

##### Trench 2

##### Test

In [ ]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Alpha-Cash-CIC-Model_All_Trench' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 2'
""" 

dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train

In [ ]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Alpha-Cash-CIC-Model_All_Trench' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 2'
;
"""
dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()


In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()

In [ ]:
df1.columns

In [ ]:
df2.columns

In [ ]:
df2.rename(columns = {
    'calc_max_age_all_contracts_snapshot':'calc_cic_max_age_all_contracts_snapshot',
    'calc_ratio_overdue_contracts_to_granted_contracts':'calc_cic_ratio_overdue_contracts_to_granted_contracts',
    'calc_ScoreRange':'calc_cic_ScoreRange',
    'calc_ln_loan_level_user_type':'calc_cic_ln_loan_level_user_type', 
    'calc_has_ever_been_overdue':'calc_cic_has_ever_been_overdue',
    'calc_latest_granted_contract_overdue_flag':'calc_cic_latest_granted_contract_overdue_flag',
    'calc_ratio_closed_over_new_granted_cnt_24M':'calc_cic_ratio_closed_over_new_granted_cnt_24M',
    'calc_ratio_risky_contracts_to_granted_contracts':'calc_cic_ratio_risky_contracts_to_granted_contracts',
    'calc_Short_and_Term_Loans_granted_contracts_cnt_24M':'calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M',
    'calc_flg_zero_non_granted_ever':'calc_cic_flg_zero_non_granted_ever', 
    'calc_CreditAvgCreditLimit':'calc_cic_CreditAvgCreditLimit',
    'calc_flg_zero_granted_ever':'calc_cic_flg_zero_granted_ever', 
    'calc_ca_cic_score':'calc_cic_ca_cic_score',
    'calc_Personal_Loans_granted_contracts_amt_24M':'calc_cic_Personal_Loans_granted_contracts_amt_24M'
    
}, inplace = True)

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()
df['aCicScore'] = pd.to_numeric(df['aCicScore'], errors='coerce')

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['aCicScore',
     'calc_cic_max_age_all_contracts_snapshot',
       'calc_cic_ratio_overdue_contracts_to_granted_contracts',
       'calc_cic_ScoreRange', 'calc_cic_ln_loan_level_user_type',
       'calc_cic_has_ever_been_overdue',
       'calc_cic_latest_granted_contract_overdue_flag',
       'calc_cic_ratio_closed_over_new_granted_cnt_24M',
       'calc_cic_ratio_risky_contracts_to_granted_contracts',
       'calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M',
       'calc_cic_flg_zero_non_granted_ever',
       'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_CreditAvgCreditLimit', 'calc_cic_flg_zero_granted_ever'
]
# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

##### Trench 3

##### Test

In [ ]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Alpha-Cash-CIC-Model_All_Trench' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 3'
""" 

dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


In [ ]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Alpha-Cash-CIC-Model_All_Trench' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 3'
;
"""
dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()


In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()


In [ ]:
df1.columns

In [ ]:
df2.columns 

In [ ]:
df2.rename(columns = {
    'calc_max_age_all_contracts_snapshot':'calc_cic_max_age_all_contracts_snapshot',
    'calc_ratio_overdue_contracts_to_granted_contracts':'calc_cic_ratio_overdue_contracts_to_granted_contracts',
    'calc_ScoreRange':'calc_cic_ScoreRange',
    'calc_ln_loan_level_user_type':'calc_cic_ln_loan_level_user_type', 
    'calc_has_ever_been_overdue':'calc_cic_has_ever_been_overdue',
    'calc_latest_granted_contract_overdue_flag':'calc_cic_latest_granted_contract_overdue_flag',
    'calc_ratio_closed_over_new_granted_cnt_24M':'calc_cic_ratio_closed_over_new_granted_cnt_24M',
    'calc_ratio_risky_contracts_to_granted_contracts':'calc_cic_ratio_risky_contracts_to_granted_contracts',
    'calc_Short_and_Term_Loans_granted_contracts_cnt_24M':'calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M',
    'calc_flg_zero_non_granted_ever':'calc_cic_flg_zero_non_granted_ever', 
    'calc_CreditAvgCreditLimit':'calc_cic_CreditAvgCreditLimit',
    'calc_flg_zero_granted_ever':'calc_cic_flg_zero_granted_ever', 
    'calc_ca_cic_score':'calc_cic_ca_cic_score',
    'calc_Personal_Loans_granted_contracts_amt_24M':'calc_cic_Personal_Loans_granted_contracts_amt_24M'
    
}, inplace = True)

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()
df['aCicScore'] = pd.to_numeric(df['aCicScore'], errors='coerce')

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['aCicScore',
     'calc_cic_max_age_all_contracts_snapshot',
       'calc_cic_ratio_overdue_contracts_to_granted_contracts',
       'calc_cic_ScoreRange', 'calc_cic_ln_loan_level_user_type',
       'calc_cic_has_ever_been_overdue',
       'calc_cic_latest_granted_contract_overdue_flag',
       'calc_cic_ratio_closed_over_new_granted_cnt_24M',
       'calc_cic_ratio_risky_contracts_to_granted_contracts',
       'calc_cic_Short_and_Term_Loans_granted_contracts_cnt_24M',
       'calc_cic_flg_zero_non_granted_ever',
       'calc_cic_Personal_Loans_granted_contracts_amt_24M',
       'calc_cic_CreditAvgCreditLimit', 'calc_cic_flg_zero_granted_ever'
]
# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

#### Alpha-Cash-Stack-Model

###### Trench 1

##### Test

In [ ]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Alpha-Cash-Stack-Model' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  prediction aStackScore,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 1'
;
"""

dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train

In [ ]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aStackScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Alpha-Cash-Stack-Model' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 1'

"""
dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()


In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()

In [ ]:
df1.columns

In [ ]:
df1.info()

In [ ]:
df2.columns

In [ ]:
df2.info()

In [ ]:
df2.rename(columns={'calc_demo_score':'calc_c_demo_score',
                     'calc_credo_score':'calc_c_credo_score' ,
                    'calc_cic_score':'calc_ca_cic_score',
                    'calc_apps_score':'calc_apps_score'
                   }, inplace = True)

##### Concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
dd.query("""select Data_selection, count(distinct digitalLoanAccountId) cnt from df_concat where calc_apps_score is not null group by 1""").to_df()

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()
df['aStackScore'] = pd.to_numeric(df['aStackScore'], errors='coerce')

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['aStackScore',  'calc_apps_score', 'calc_c_demo_score',   'calc_c_credo_score', 'calc_ca_cic_score']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

##### Trench 2

##### Test

In [ ]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Alpha-Cash-Stack-Model' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  prediction aStackScore,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 2'
;
"""

dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train 

In [ ]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aStackScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Alpha-Cash-Stack-Model' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 2'

"""
dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()


In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()

In [ ]:
df1.info()

In [ ]:
df2.info()

In [ ]:
df2.rename(columns={'calc_demo_score':'calc_c_demo_score',
                     'calc_credo_score':'calc_c_credo_score' ,
                    'calc_cic_score':'calc_ca_cic_score',
                    'calc_apps_score':'calc_apps_score'
                   }, inplace = True)

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()
df['aStackScore'] = pd.to_numeric(df['aStackScore'], errors='coerce')

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['aStackScore',  'calc_apps_score', 'calc_c_demo_score',   'calc_c_credo_score', 'calc_ca_cic_score']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

In [ ]:
# sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_psi_v4"""
# dfd = client.query(sq).to_dataframe()

##### Trench 3

##### Test

In [ ]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Alpha-Cash-Stack-Model' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  prediction aStackScore,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 3'
;
"""

dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train 

In [ ]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aStackScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Alpha-Cash-Stack-Model' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 3'

"""
dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()


In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()

In [ ]:
df1.info()

In [ ]:
df2.info()

In [ ]:
df2.rename(columns={'calc_demo_score':'calc_c_demo_score',
                     'calc_credo_score':'calc_c_credo_score' ,
                    'calc_cic_score':'calc_ca_cic_score',
                    'calc_apps_score':'calc_apps_score'
                   }, inplace = True)

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()
df['aStackScore'] = pd.to_numeric(df['aStackScore'], errors='coerce')

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['aStackScore',  'calc_apps_score', 'calc_c_demo_score',   'calc_c_credo_score', 'calc_ca_cic_score']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

In [ ]:
# sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_psi_v4"""
# dfd = client.query(sq).to_dataframe()
# dfd.groupby(['trenchCategory','Month'])['modelDisplayName'].count()

#### Beta-Cash-Demo-Model

##### Trench 1

##### Test

In [ ]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-Demo-Model_All_Trench' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  prediction Beta_Cash_Demo_Score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 1'
;
"""
dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train

In [ ]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-Demo-Model_All_Trench' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  prediction Beta_Cash_Demo_Score,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 1'
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()


In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()

In [ ]:
df1.info()

In [ ]:
df2.info()

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
df_concat = df_concat.drop(columns=['calc_Beta_Cash_Demo_Score', 'calc_ln_fspd30_flag', 'calc_ln_os_type', 'calc_ln_mature_fspd30_flag', 'calc_ln_appln_submit_datetime']).copy()
df_concat.columns = df_concat.columns.str.replace('calc_', '', regex=False)
df_concat.columns

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()
df['Beta_Cash_Demo_Score'] = pd.to_numeric(df['Beta_Cash_Demo_Score'], errors='coerce')
df.columns

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['Beta_Cash_Demo_Score', 'ln_vas_opted_flag', 'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin', 'ln_marital_status', 'ln_doc_type_rolled',
       'ln_education_level', 'ln_ref2_type', 'ln_email_primary_domain',
       'ln_province_bin']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

In [ ]:
# sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_psi_v4"""
# dfd = client.query(sq).to_dataframe()
# dfd.groupby(['trenchCategory','Month'])['modelDisplayName'].count()

##### Trench 2

#### Test

In [ ]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-Demo-Model_All_Trench' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  prediction Beta_Cash_Demo_Score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 2'
;
"""
dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train

In [ ]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-Demo-Model_All_Trench' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  prediction Beta_Cash_Demo_Score,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 2'
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()


In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()

In [ ]:
df1.info()

In [ ]:
df2.info()

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
df_concat = df_concat.drop(columns=['calc_Beta_Cash_Demo_Score', 'calc_ln_fspd30_flag', 'calc_ln_os_type', 'calc_ln_mature_fspd30_flag', 'calc_ln_appln_submit_datetime']).copy()
df_concat.columns = df_concat.columns.str.replace('calc_', '', regex=False)
df_concat.columns

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()
df['Beta_Cash_Demo_Score'] = pd.to_numeric(df['Beta_Cash_Demo_Score'], errors='coerce')
df.columns

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['Beta_Cash_Demo_Score', 'ln_vas_opted_flag', 'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin', 'ln_marital_status', 'ln_doc_type_rolled',
       'ln_education_level', 'ln_ref2_type', 'ln_email_primary_domain',
        'ln_province_bin']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

In [ ]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_psi_v4"""
dfd = client.query(sq).to_dataframe()
dfd.groupby(['trenchCategory','Month'])['modelDisplayName'].count()

##### Trench 3

#### Test

In [ ]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-Demo-Model_All_Trench' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  prediction Beta_Cash_Demo_Score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 3'
;
"""
dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train

In [ ]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-Demo-Model_All_Trench' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  prediction Beta_Cash_Demo_Score,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 3'
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()


In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()

In [ ]:
df1.info()

In [ ]:
df2.info()

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
df_concat = df_concat.drop(columns=['calc_Beta_Cash_Demo_Score']).copy()
df_concat.columns = df_concat.columns.str.replace('calc_', '', regex=False)
df_concat.columns

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()
df['Beta_Cash_Demo_Score'] = pd.to_numeric(df['Beta_Cash_Demo_Score'], errors='coerce')
df.columns

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['Beta_Cash_Demo_Score', 'ln_vas_opted_flag', 'ln_self_dec_income', 'ln_age',
       'ln_source_funds_new_bin', 'ln_loan_level_user_type',
       'ln_industry_new_cat_bin', 'ln_marital_status', 'ln_doc_type_rolled',
       'ln_education_level', 'ln_ref2_type', 'ln_email_primary_domain',
        'ln_province_bin']

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

In [ ]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_psi_v4"""
dfd = client.query(sq).to_dataframe()
dfd.groupby(['trenchCategory','Month'])['modelDisplayName'].count()

#### Beta-Cash-AppScore-Model

##### Trench 1

##### Test

In [ ]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-AppScore-Model_Trench1' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  SAFE_CAST(JSON_VALUE(r.prediction_clean, "$.combined_score") AS Float64) AS beta_cash_app_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 1'
;"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train

In [ ]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-AppScore-Model_Trench1' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  prediction beta_cash_app_score,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 1'
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()

In [ ]:
df1.info()

In [ ]:
df2.info()

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
df_concat = df_concat.drop(columns=['prediction', 'calc_app_median_time_bw_installed_mins_ever', 'calc_app_avg_time_bw_installed_mins_ever', 'calc_beta_cash_app_score']).copy()
df_concat.columns = df_concat.columns.str.replace('calc_', '', regex=False)
df_concat.columns

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()
df.info()

In [ ]:
df['beta_cash_app_score'] = pd.to_numeric(df['beta_cash_app_score'], errors='coerce')
df.columns

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d'
]

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

In [ ]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_psi_v4"""
dfd = client.query(sq).to_dataframe()
dfd.groupby(['trenchCategory','Month'])['modelDisplayName'].count()

##### Trench 2

##### Test

In [ ]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-AppScore-Model_Trench2' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  SAFE_CAST(JSON_VALUE(r.prediction_clean, "$.combined_score") AS Float64) AS beta_cash_app_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 2'
;"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train

In [ ]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-AppScore-Model_Trench2' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  prediction beta_cash_app_score,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 2'
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()

In [ ]:
df1.info()

In [ ]:
df2.info()

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
df_concat = df_concat.drop(columns=['prediction', 'calc_app_median_time_bw_installed_mins_ever', 'calc_app_avg_time_bw_installed_mins_ever', 'calc_beta_cash_app_score']).copy()
df_concat.columns = df_concat.columns.str.replace('calc_', '', regex=False)
df_concat.columns

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()
df.info()

In [ ]:
df['beta_cash_app_score'] = pd.to_numeric(df['beta_cash_app_score'], errors='coerce')
df.columns

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['beta_cash_app_score',
       'app_cnt_health_and_fitness_ever', 'app_cnt_shopping_ever',
       'app_cnt_crypto_ever', 'app_cnt_driver_ever', 'app_cnt_payday_180d',
       'app_cnt_gambling_180d', 'app_avg_time_bw_installed_mins_3d',
       'app_median_time_bw_installed_mins_3d'
]

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

In [ ]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_psi_v4"""
dfd = client.query(sq).to_dataframe()
dfd.groupby(['trenchCategory','Month'])['modelDisplayName'].count()

##### Trench 3

##### Test

In [ ]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-AppScore-Model_Trench3' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  SAFE_CAST(JSON_VALUE(r.prediction_clean, "$.combined_score") AS Float64) AS beta_cash_app_score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 3'
;"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train

In [ ]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-AppScore-Model_Trench3' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  prediction beta_cash_app_score,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 3'
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()

In [ ]:
df1.info()

In [ ]:
df2.info()

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
df_concat = df_concat.drop(columns=['prediction', 'calc_beta_cash_app_score']).copy()
df_concat.columns = df_concat.columns.str.replace('calc_', '', regex=False)
df_concat.columns

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()
df.info()

In [ ]:
df['beta_cash_app_score'] = pd.to_numeric(df['beta_cash_app_score'], errors='coerce')
df.columns

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['beta_cash_app_score', 'app_cnt_health_and_fitness_ever', 'app_cnt_productivity_ever',
       'app_cnt_rated_for_18plus_ever', 'app_cnt_books_and_reference_ever',
       'app_cnt_gaming_180d', 'app_cnt_absence_tag_365d',
       'app_last_payday_install_to_apply_days',
       'app_cnt_absence_tag_365d_binned', 'app_cnt_gaming_180d_binned',
       'app_cnt_productivity_ever_binned',
       'app_cnt_rated_for_18plus_ever_binned',
       'app_cnt_health_and_fitness_ever_binned',
       'app_cnt_books_and_reference_ever_binned',
       'app_last_payday_install_to_apply_days_binned', 'calc_ln_user_type'
]

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

In [ ]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_psi_v4"""
dfd = client.query(sq).to_dataframe()
dfd.groupby(['trenchCategory','Month'])['modelDisplayName'].count()

#### Beta-Cash-Stack-Model Trench1

##### Trench 1

##### Test

In [ ]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction Beta_cash_stack_score
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-Stack-Model_Trench1' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 1'
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train

In [ ]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction Beta_cash_stack_score
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-Stack-Model_Trench1' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 1'
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()

In [ ]:
df1.info()

In [ ]:
df2.info()

In [ ]:
df2.rename(columns={'calc_apps_score':'calc_app_score'}, inplace = True)

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
# df_concat = df_concat.drop(columns=['prediction', 'calc_app_median_time_bw_installed_mins_ever', 'calc_app_avg_time_bw_installed_mins_ever', 'calc_beta_cash_app_score']).copy()
df_concat.columns = df_concat.columns.str.replace('calc_', '', regex=False)
df_concat.columns

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()
df.info()

In [ ]:
df['Beta_cash_stack_score'] = pd.to_numeric(df['Beta_cash_stack_score'], errors='coerce')
df.columns

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['Beta_cash_stack_score',
       'demo_score', 'credo_score', 'app_score'
]

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

In [ ]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_psi_v4"""
dfd = client.query(sq).to_dataframe()
dfd.groupby(['trenchCategory','Month'])['modelDisplayName'].count()

##### Trench 2

##### Test

In [ ]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction Beta_cash_stack_score
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-Stack-Model_Trench2' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 2'
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train

In [ ]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction Beta_cash_stack_score
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-Stack-Model_Trench2' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 2'
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()

In [ ]:
df1.info()

In [ ]:
df2.info()

In [ ]:
df2.rename(columns={'calc_apps_score':'calc_app_score'}, inplace = True)

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
# df_concat = df_concat.drop(columns=['prediction', 'calc_app_median_time_bw_installed_mins_ever', 'calc_app_avg_time_bw_installed_mins_ever', 'calc_beta_cash_app_score']).copy()
df_concat.columns = df_concat.columns.str.replace('calc_', '', regex=False)
df_concat.columns

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()
df.info()

In [ ]:
df['Beta_cash_stack_score'] = pd.to_numeric(df['Beta_cash_stack_score'], errors='coerce')
df.columns

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['Beta_cash_stack_score',
       'demo_score', 'credo_score', 'app_score'
]

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

In [ ]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_psi_v4"""
dfd = client.query(sq).to_dataframe()
dfd.groupby(['trenchCategory','Month'])['modelDisplayName'].count()

##### Trench 3

##### Test

In [ ]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction Beta_cash_stack_score
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-Stack-Model_Trench3' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 3'
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df1 = expanded_df.copy()


##### Train

In [ ]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,modelDisplayName,modelVersionId,start_time,end_time,prediction,trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction Beta_cash_stack_score
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type loanType,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'Beta-Cash-Stack-Model_Trench3' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where trenchCategory = 'Trench 3'
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

In [ ]:
# dfd.groupby(['Application_month', 'trenchCategory'])['digitalLoanAccountId'].nunique()

In [ ]:
# Expand the calcFeatures column
expanded_df = expand_calc_features(dfd)

# Display the result
print(f"Original columns: {dfd.shape[1]}")
print(f"Expanded columns: {expanded_df.shape[1]}")
df2 = expanded_df.copy()

In [ ]:
df1.info()

In [ ]:
df2.info()

In [ ]:
df2.rename(columns={'calc_apps_score':'calc_app_score'}, inplace = True)

##### concatenate

In [ ]:
df_concat = pd.concat([df1, df2], ignore_index=True)
df_concat.info()

In [ ]:
# df_concat = df_concat.drop(columns=['prediction', 'calc_app_median_time_bw_installed_mins_ever', 'calc_app_avg_time_bw_installed_mins_ever', 'calc_beta_cash_app_score']).copy()
df_concat.columns = df_concat.columns.str.replace('calc_', '', regex=False)
df_concat.columns

In [ ]:
df = df_concat.drop(columns=['calcFeatures']).copy()
df.info()

In [ ]:
df['Beta_cash_stack_score'] = pd.to_numeric(df['Beta_cash_stack_score'], errors='coerce')
df.columns

##### PSI calculation

In [ ]:
df = df.copy()

# Define feature list
feature_list = ['Beta_cash_stack_score',
       'demo_score', 'credo_score', 'app_score'
]

# Define segment columns
segment_columns = ['new_loan_type','osType', 'loan_product_type']
# Calculate month-on-month PSI
psi_results = calculate_month_on_month_psi(df, feature_list, segment_columns)
psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
psi_results['Model_Name'] = df['Model_Name'].iloc[0]
psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
psi_results = psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
                           'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value', 'Month',
                           'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
                           'Expected_Percentage', 'Actual_Percentage', 'PSI']].copy()

# # Calculate bin-level PSI
# bin_psi_results = calculate_bin_level_psi(df, feature_list, segment_columns)
# bin_psi_results['modelDisplayName'] = df['modelDisplayName'].iloc[0]
# bin_psi_results['Model_Name'] = df['Model_Name'].iloc[0]
# bin_psi_results['modelVersionId'] = df['modelVersionId'].iloc[0]
# bin_psi_results['trenchCategory'] = df['trenchCategory'].iloc[0]
# bin_psi_results = bin_psi_results[['modelDisplayName', 'Model_Name', 'modelVersionId', 'trenchCategory',
#                                    'Feature', 'Feature_Type', 'Segment_Column', 'Segment_Value',
#                                     'Month', 'Base_Month', 'Current_Month', 'Base_Count', 'Actual_Count',
#                                     'Bin', 'Bin_Range', 'Base_Percentage', 'Actual_Percentage', 'Bin_PSI']].copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_psi_v4"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


# # Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_income_estimation_model_csi_v4"
# job_config = bigquery.LoadJobConfig(
#     write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
# )
# job = client.load_table_from_dataframe(bin_psi_results, table_id, job_config=job_config)
# job.result()  # Wait for the job to complete

In [ ]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_psi_v4"""
dfd = client.query(sq).to_dataframe()
dfd.groupby(['trenchCategory','Month'])['modelDisplayName'].count()

# End